In [1]:
#!/usr/bin/env python
# coding: utf-8

# -*- coding: utf-8 -*-
"""
Created on Sat Sep  5 20:36:16 2020

@author: consultant138
"""
import os
#os.chdir('D:\\ViteosModel')


import numpy as np
import pandas as pd
#from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from tqdm import tqdm
import pickle
import datetime as dt
import sys
from ViteosMongoDB import  ViteosMongoDB_Class as mngdb
from datetime import datetime,date,timedelta
from pandas.io.json import json_normalize
import dateutil.parser
from difflib import SequenceMatcher
import pprint
import json
from pandas import merge
import re

import dask.dataframe as dd
import glob
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from dateutil.parser import parse
import operator
import itertools
from sklearn.feature_extraction.text import CountVectorizer

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

from fuzzywuzzy import fuzz
import random
import decimal

cols = ['Currency','Account Type','Accounting Net Amount',
#'Accounting Net Amount Difference','Accounting Net Amount Difference Absolute ',
'Task ID', 'Source Combination Code',
'Activity Code','Age','Age WK',
'Asset Type Category','Base Currency','Base Net Amount','Bloomberg_Yellow_Key',
'B-P Net Amount',
#'B-P Net Amount Difference','B-P Net Amount Difference Absolute',
'BreakID',
'Business Date','Cancel Amount','Cancel Flag','CUSIP','Custodian',
'Custodian Account',
'Derived Source','Description','Department','ExpiryDate','ExternalComment1','ExternalComment2',
'ExternalComment3','Fund','FX Rate','Interest Amount','InternalComment1','InternalComment2',
'InternalComment3','Investment Type','Is Combined Data','ISIN','Keys',
'Mapped Custodian Account','Net Amount Difference','Net Amount Difference Absolute','Non Trade Description',
'OTE Custodian Account',
#'Predicted Action','Predicted Status','Prediction Details',
'Price','Prime Broker',
'Quantity','SEDOL','Settle Date','SPM ID','Status','Strike Price',
'System Comments','Ticker','Trade Date','Trade Expenses','Transaction Category','Transaction ID','Transaction Type',
'Underlying Cusip','Underlying Investment ID','Underlying ISIN','Underlying Sedol','Underlying Ticker','Source Combination','_ID']
#'UnMapped']

add = ['ViewData.Side0_UniqueIds', 'ViewData.Side1_UniqueIds',
      # 'MetaData.0._RecordID','MetaData.1._RecordID',
       'ViewData.Task Business Date']





new_cols = ['ViewData.' + x for x in cols] + add

common_cols = ['ViewData.Accounting Net Amount', 'ViewData.Age',
'ViewData.Age WK', 'ViewData.Asset Type Category',
'ViewData.B-P Net Amount', 'ViewData.Base Net Amount','ViewData.CUSIP', 
 'ViewData.Cancel Amount',
       'ViewData.Cancel Flag',
#'ViewData.Commission',
        'ViewData.Currency', 'ViewData.Custodian',
       'ViewData.Custodian Account',
       'ViewData.Description','ViewData.Department', 'ViewData.ExpiryDate', 'ViewData.Fund',
       'ViewData.ISIN',
       'ViewData.Investment Type',
      # 'ViewData.Keys',
       'ViewData.Mapped Custodian Account',
       'ViewData.Net Amount Difference',
       'ViewData.Net Amount Difference Absolute',
        #'ViewData.OTE Ticker',
        'ViewData.Price',
       'ViewData.Prime Broker', 'ViewData.Quantity',
       'ViewData.SEDOL', 'ViewData.SPM ID', 'ViewData.Settle Date',
       
  #  'ViewData.Strike Price',
               'Date',
       'ViewData.Ticker', 'ViewData.Trade Date',
       'ViewData.Transaction Category',
       'ViewData.Transaction Type', 'ViewData.Underlying Cusip',
       'ViewData.Underlying ISIN',
       'ViewData.Underlying Sedol','filter_key','ViewData.Status','ViewData.BreakID',
              'ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','ViewData._ID']

model_cols = [
            'SideA.ViewData.B-P Net Amount', 
              #'SideA.ViewData.Cancel Flag', 
              #'SideA.new_desc_cat',
             # 'SideA.ViewData.Description',
             # 'SideA.ViewData.Department',
   
    
              
             # 'SideA.ViewData.Price',
             # 'SideA.ViewData.Quantity',
             #'SideA.ViewData.Investment Type', 
              #'SideA.ViewData.Asset Type Category', 
              'SideB.ViewData.Accounting Net Amount', 
              #'SideB.ViewData.Cancel Flag', 
             # 'SideB.ViewData.Description',
              # 'SideB.ViewData.Department',
              
             # 'SideB.ViewData.Price',
             # 'SideB.ViewData.Quantity',
             # 'SideB.new_desc_cat',
             # 'SideB.ViewData.Investment Type', 
              #'SideB.ViewData.Asset Type Category', 
              'Trade_Date_match', 'Settle_Date_match', 
                'Amount_diff_2', 
              'Trade_date_diff', 'Settle_date_diff', 'SideA.ISIN_NA', 'SideB.ISIN_NA', 
             # 'ViewData.Combined Fund',
              'ViewData.Combined Transaction Type', 'Combined_Desc','Combined_TType',
             # 'SideA.TType', 'SideB.TType', 
              'abs_amount_flag',
    'tt_map_flag', 
              'All_key_nan','new_key_match', 'new_pb1',
              'SideB.Date','SideA.ViewData.Settle Date','SideB.ViewData.Settle Date',
            'SideA.ViewData._ID', 'SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds',
              'SideB.ViewData.Status', 'SideB.ViewData.BreakID_B_side',
              'SideA.ViewData.Status', 'SideA.ViewData.BreakID_A_side'] 
              #'label']

model_cols_2 =[#'SideA.ViewData.B-P Net Amount', 
              #'SideA.ViewData.Cancel Flag', 
              #'SideA.new_desc_cat',
             # 'SideA.ViewData.Description',
             # 'SideA.ViewData.Department',
   
    
              
             # 'SideA.ViewData.Price',
             # 'SideA.ViewData.Quantity',
             #'SideA.ViewData.Investment Type', 
              #'SideA.ViewData.Asset Type Category', 
              #'SideB.ViewData.Accounting Net Amount', 
              #'SideB.ViewData.Cancel Flag', 
             # 'SideB.ViewData.Description',
              # 'SideB.ViewData.Department',
              
             # 'SideB.ViewData.Price',
             # 'SideB.ViewData.Quantity',
             # 'SideB.new_desc_cat',
             # 'SideB.ViewData.Investment Type', 
              #'SideB.ViewData.Asset Type Category', 
              'Trade_Date_match', 'Settle_Date_match', 
              #  'Amount_diff_2', 
              'Trade_date_diff', 'Settle_date_diff', 'SideA.ISIN_NA', 'SideB.ISIN_NA', 
             # 'ViewData.Combined Fund',
              'ViewData.Combined Transaction Type', 'Combined_Desc','Combined_TType',
             # 'SideA.TType', 'SideB.TType', 
              'abs_amount_flag',
    'tt_map_flag', 
              'All_key_nan','new_key_match', 'new_pb1',
              'SideB.Date','SideA.ViewData.Settle Date','SideB.ViewData.Settle Date',
            'SideA.ViewData._ID', 'SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds',
              'SideB.ViewData.Status', 'SideB.ViewData.BreakID_B_side',
              'SideA.ViewData.Status', 'SideA.ViewData.BreakID_A_side'] 
              #'label']

#### Closed break functions - Begin #### 

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def dictionary_exclude_keys(fun_dict, fun_keys_to_exclude):
    return {x: fun_dict[x] for x in fun_dict if x not in fun_keys_to_exclude}

def write_dict_at_top(fun_filename, fun_dict_to_add):
    with open(fun_filename, 'r+') as f:
        fun_existing_content = f.read()
        f.seek(0, 0)
        f.write(json.dumps(fun_dict_to_add, indent = 4))
        f.write('\n')
        f.write(fun_existing_content)

def normalize_bp_acct_col_names(fun_df):
    bp_acct_col_names_mapping_dict = {
                                      'ViewData.Cust Net Amount' : 'ViewData.B-P Net Amount',
                                      'ViewData.Cust Net Amount Difference' : 'ViewData.B-P Net Amount Difference',
                                      'ViewData.Cust Net Amount Difference Absolute' : 'ViewData.B-P Net Amount Difference Absolute',
                                      'ViewData.CP Net Amount' : 'ViewData.B-P Net Amount',
                                      'ViewData.CP Net Amount Difference' : 'ViewData.B-P Net Amount Difference',
                                      'ViewData.CP Net Amount Difference Absolute' : 'ViewData.B-P Net Amount Difference Absolute',
                                      'ViewData.PMSVendor Net Amount' : 'ViewData.Accounting Net Amount'
                                        }
    fun_df.rename(columns = bp_acct_col_names_mapping_dict, inplace = True)
    return(fun_df)



# M X M and N X N architecture for closed break prediction
def closed_cols():
    cols_for_closed_list = ['Status','Source Combination','Mapped Custodian Account',
                   'Accounting Currency','B-P Currency', 
                   'Transaction ID','Transaction Type','Description','Investment ID',
                   'Accounting Net Amount','B-P Net Amount', 
                   'InternalComment2','Custodian','Fund']
    cols_for_closed_list = ['ViewData.' + x for x in cols_for_closed_list]
    cols_for_closed_x_list = [x + '_x' for x in cols_for_closed_list] + ['ViewData.Side0_UniqueIds_x','ViewData.Side1_UniqueIds_x']
    cols_for_closed_y_list = [x + '_y' for x in cols_for_closed_list] + ['ViewData.Side0_UniqueIds_y','ViewData.Side1_UniqueIds_y']
    cols_for_closed_x_y_list = cols_for_closed_x_list + cols_for_closed_y_list
    return({
            'cols_for_closed' : cols_for_closed_list,
            'cols_for_closed_x' : cols_for_closed_x_list,
            'cols_for_closed_y' : cols_for_closed_y_list,
            'cols_for_closed_x_y' : cols_for_closed_x_y_list
            })

def cleaned_meo(#fun_filepath_meo, 
                fun_meo_df):
#    meo = pd.read_csv(fun_filepath_meo)           .drop_duplicates()           .reset_index()           .drop('index',1)
    meo = fun_meo_df
    meo = normalize_bp_acct_col_names(fun_df = meo)
    
#    Commened out below line on 26-11-2020 to exclude SPM from closed coverage, and added the line below the commened line
#    meo = meo[~meo['ViewData.Status'].isin(['SMT','HST', 'OC', 'CT', 'Archive','SMR'])]
    meo = meo[~meo['ViewData.Status'].isin(['SPM','SMT','HST', 'OC', 'CT', 'Archive','SMR'])] 
    meo = meo[~meo['ViewData.Status'].isnull()]           .reset_index()           .drop('index',1)
    
    meo['Date'] = pd.to_datetime(meo['ViewData.Task Business Date'])
    meo = meo[~meo['Date'].isnull()]           .reset_index()           .drop('index',1)
    meo['Date'] = pd.to_datetime(meo['Date']).dt.date
    meo['Date'] = meo['Date'].astype(str)

    meo['ViewData.Side0_UniqueIds'] = meo['ViewData.Side0_UniqueIds'].astype(str)
    meo['ViewData.Side1_UniqueIds'] = meo['ViewData.Side1_UniqueIds'].astype(str)

    meo['flag_side0'] = meo.apply(lambda x: len(x['ViewData.Side0_UniqueIds'].split(',')), axis=1)
    meo['flag_side1'] = meo.apply(lambda x: len(x['ViewData.Side1_UniqueIds'].split(',')), axis=1)

    meo.loc[meo['ViewData.Side0_UniqueIds']=='nan','flag_side0'] = 0
    meo.loc[meo['ViewData.Side1_UniqueIds']=='nan','flag_side1'] = 0

    meo.loc[meo['ViewData.Side0_UniqueIds']=='None','flag_side0'] = 0
    meo.loc[meo['ViewData.Side1_UniqueIds']=='None','flag_side1'] = 0
   
    meo['ViewData.BreakID'] = meo['ViewData.BreakID'].astype(int)
    meo = meo[meo['ViewData.BreakID']!=-1]           .reset_index()           .drop('index',1)
          
    meo['Side_0_1_UniqueIds'] = meo['ViewData.Side0_UniqueIds'].astype(str) +                                 meo['ViewData.Side1_UniqueIds'].astype(str)
                                
    meo = meo.sort_values(by=['ViewData.Transaction ID','ViewData.Transaction Type'],ascending = False)
    return(meo)
    
def cleaned_aua(fun_filepath_aua):
    aua = pd.read_csv(fun_filepath_aua)       .drop_duplicates()       .reset_index()       .drop('index',1)       .sort_values(by=['ViewData.Transaction ID','ViewData.Transaction Type'],ascending = False)

    aua = normalize_bp_acct_col_names(fun_df = aua)

    
    aua['Side_0_1_UniqueIds'] = aua['ViewData.Side0_UniqueIds'].astype(str) +                                 aua['ViewData.Side1_UniqueIds'].astype(str)
    
    return(aua)

def Acct_MEO_combination_file(fun_side, fun_cleaned_meo_df):
    if(fun_side == 'PB' or fun_side == 'BP' or fun_side == 'B-P' or fun_side == 'Prime Broker'):
        side_meo = fun_cleaned_meo_df[(fun_cleaned_meo_df['flag_side1'] >= 1) & (fun_cleaned_meo_df['flag_side0'] == 0)]
#        Currency_col_name = 'ViewData.B-P Currency'
    elif(fun_side == 'Acct' or fun_side == 'Accounting'):
        side_meo = fun_cleaned_meo_df[(fun_cleaned_meo_df['flag_side1'] == 0) & (fun_cleaned_meo_df['flag_side0'] >= 1)]
#        Currency_col_name = 'ViewData.Accounting Currency'
    else:
        print('The only options for side are on of the following : ')
        print('For Prime Broker side, the options are PB or BP or B-P or Prime Broker')
        print('For Accounting side, the options are Acct or Accounting')
        raise ValueError('Exiting function because fun_side argument was not from the accepted set of parameter values')
    
    side_meo['filter_key'] = side_meo['ViewData.Source Combination'].astype(str) +                          side_meo['ViewData.Mapped Custodian Account'].astype(str) +                          side_meo['ViewData.Currency'].astype(str)
        
    side_meo_training_df =[]
    for key in (list(np.unique(np.array(list(side_meo['filter_key'].values))))):
        side_meo_filter_slice = side_meo[side_meo['filter_key']==key]
        if side_meo_filter_slice.empty == False:
    
            side_meo_filter_slice = side_meo_filter_slice.reset_index()
            side_meo_filter_slice = side_meo_filter_slice.drop('index', 1)
    
            side_meo_filter_joined = pd.merge(side_meo_filter_slice, side_meo_filter_slice, on='filter_key')
            side_meo_training_df.append(side_meo_filter_joined)
    return(pd.concat(side_meo_training_df))
    
def identifying_closed_breaks_from_Trans_type(fun_side, fun_transaction_type_list, fun_side_meo_combination_df, fun_setup_code_crucial):
    if(fun_side == 'PB' or fun_side == 'BP' or fun_side == 'B-P' or fun_side == 'Prime Broker'):
        Net_amount_col_name_list = ['ViewData.B-P Net Amount_' + x for x in ['x','y']]
        Side_0_1_UniqueIds_col_name_list = ['ViewData.Side1_UniqueIds_' + x for x in ['x','y']]
    elif(fun_side == 'Acct' or fun_side == 'Accounting'):
        Net_amount_col_name_list = ['ViewData.Accounting Net Amount_' + x for x in ['x','y']]
        Side_0_1_UniqueIds_col_name_list = ['ViewData.Side0_UniqueIds_' + x for x in ['x','y']]
    else:
        print('The only options for side are on of the following : ')
        print('For Prime Broker side, the options are PB or BP or B-P or Prime Broker')
        print('For Accounting side, the options are Acct or Accounting')
        raise ValueError('Exiting function because fun_side argument was not from the accepted set of parameter values')        
    
    if(fun_setup_code_crucial == '379'):
        Transaction_type_closed_break_df =             fun_side_meo_combination_df[                     (fun_side_meo_combination_df['ViewData.Transaction Type_x'].astype(str).isin(fun_transaction_type_list)) &                     (fun_side_meo_combination_df['ViewData.Transaction Type_y'].astype(str).isin(fun_transaction_type_list)) &                     (abs(fun_side_meo_combination_df[Net_amount_col_name_list[0]]).astype(str) == abs(fun_side_meo_combination_df[Net_amount_col_name_list[1]]).astype(str)) &                     (fun_side_meo_combination_df[Side_0_1_UniqueIds_col_name_list[0]].astype(str) != fun_side_meo_combination_df[Side_0_1_UniqueIds_col_name_list[1]].astype(str))                     ]
    return(set(
                Transaction_type_closed_break_df['ViewData.Side0_UniqueIds_x'].astype(str) + \
                Transaction_type_closed_break_df['ViewData.Side1_UniqueIds_x'].astype(str)
               ))

def closed_breaks_captured_mode(fun_aua_df, fun_transaction_type, fun_captured_closed_breaks_set, fun_mode):
    if(fun_transaction_type != 'All_Closed_Breaks'):
        aua_df = fun_aua_df[(fun_aua_df['ViewData.Status'] == 'UCB') &                             (fun_aua_df['ViewData.Transaction Type'] == fun_transaction_type)]
    else:
        aua_df = fun_aua_df[(fun_aua_df['ViewData.Status'] == 'UCB')]
        
    aua_side_0_1_UniqueIds_set = set(aua_df['ViewData.Side0_UniqueIds'].astype(str) +                                  aua_df['ViewData.Side1_UniqueIds'].astype(str))
    if(fun_mode == 'Correctly_Captured_In_AUA'):
        list_to_return = list(aua_side_0_1_UniqueIds_set & fun_captured_closed_breaks_set)
    elif(fun_mode == 'Not_Captured_In_AUA'):
        list_to_return = list(aua_side_0_1_UniqueIds_set - fun_captured_closed_breaks_set)
    elif(fun_mode == 'Over_Captured_In_AUA'):
        list_to_return = list(fun_captured_closed_breaks_set - aua_side_0_1_UniqueIds_set)
    return(list_to_return)

def update_dict_to_output_breakids_number_pct(fun_dict, fun_aua_df, fun_loop_transaction_type, fun_count, fun_Side_0_1_UniqueIds_list):
    mode_type_list = ['Correctly_Captured_In_AUA','Not_Captured_In_AUA','Over_Captured_In_AUA']
    for mode_type in mode_type_list:
#    if(fun_loop_transaction_type != 'All_Closed_Breaks'):
        fun_dict[fun_loop_transaction_type][mode_type + '_BreakIDs_in_AUA'] = list(set(            fun_aua_df[fun_aua_df['Side_0_1_UniqueIds'].isin(                     closed_breaks_captured_mode(fun_aua_df = fun_aua_df,                                         fun_transaction_type = fun_loop_transaction_type,                                         fun_captured_closed_breaks_set = set(fun_Side_0_1_UniqueIds_list),                                         fun_mode = mode_type))]                    ['ViewData.BreakID']))
    
        fun_total_number = len(                             fun_dict[fun_loop_transaction_type][mode_type + '_BreakIDs_in_AUA'])
        
        fun_dict[fun_loop_transaction_type][mode_type + '_Total_Number'] = len(                             fun_dict[fun_loop_transaction_type][mode_type + '_BreakIDs_in_AUA'])
        
        if(fun_count != 0):
            
            fun_dict[fun_loop_transaction_type][mode_type + '_Percentage'] = fun_total_number/fun_count#\
#                                 fun_dict[fun_loop_transaction_type][mode_type + '_Total_Number']/fun_count
        
        else:
            fun_dict[fun_loop_transaction_type][mode_type + '_Percentage'] = fun_loop_transaction_type + ' not found in Closed breaks of AUA'
    return(fun_dict)

def closed_daily_run(fun_setup_code, 
                     fun_date, 
                     fun_meo_df_daily_run#,
#                     fun_main_filepath_meo, 
#                     fun_main_filepath_aua
                     ):
    setup_val = fun_setup_code
    main_meo = cleaned_meo(fun_meo_df = fun_meo_df_daily_run)#, fun_filepath_meo = fun_main_filepath_meo
    
    BP_meo_training_df = Acct_MEO_combination_file(fun_side = 'PB', \
                                                   fun_cleaned_meo_df = main_meo)
    
    Acct_meo_training_df = Acct_MEO_combination_file(fun_side = 'Acct', \
                                                     fun_cleaned_meo_df = main_meo)

#    main_aua = cleaned_aua(fun_filepath_aua = fun_main_filepath_aua)
    
    if(fun_setup_code == '379'):
        Transaction_Type_dict = {
                                'Interest BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['Interest'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Interest Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Interest'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'STIF Interest BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['STIF Interest'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'STIF Interest Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['STIF Interest'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Buy BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['Buy'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Buy Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Buy'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Sell BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['Sell'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Sell Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Sell'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'ForwardFX BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['ForwardFX'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'ForwardFX Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['ForwardFX'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Internal Trans' : {'side' : 'PB',
                                           'Transaction_Type' : ['Internal Trans'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Withdraw' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Withdraw'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Deposit' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Deposit'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Redemption' : {'side' : 'PB',
                                           'Transaction_Type' : ['Redemption'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Subscription' : {'side' : 'PB',
                                           'Transaction_Type' : ['Redemption'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Incoming Wire' : {'side' : 'PB',
                                           'Transaction_Type' : ['Incoming Wire'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Transfer' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Transfer'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Withdrawal BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['Withdrawal'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Withdrawal Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Withdrawal'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Revenue' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Revenue'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Pay Down' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Pay Down'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Over & Short' : {'side' : 'PB',
                                           'Transaction_Type' : ['Over & Short'],
                                           'Side_meo_training_df' : BP_meo_training_df}
                                }

    print(os.getcwd())
    os.chdir('D:\\ViteosModel\\Closed')
    print(os.getcwd())
    
    filepath_stdout = fun_setup_code + '_closed_run_date_' + str(fun_date) + '_timestamp_' + str(datetime.now().strftime("%d_%m_%Y_%H_%M")) + '.txt'
    orig_stdout = sys.stdout
    f = open(filepath_stdout, 'w')
    sys.stdout = f
    
    Side_0_1_UniqueIds_closed_all_list = []
    for Transaction_type in Transaction_Type_dict:

        Side_0_1_UniqueIds_for_Transaction_type = identifying_closed_breaks_from_Trans_type(fun_side = Transaction_Type_dict.get(Transaction_type).get('side'), \
                                                                                            fun_transaction_type_list = Transaction_Type_dict.get(Transaction_type).get('Transaction_Type'), \
                                                                                            fun_side_meo_combination_df = Transaction_Type_dict.get(Transaction_type).get('Side_meo_training_df'), \
                                                                                            fun_setup_code_crucial = setup_val)

#        count_closed_breaks_for_transaction_type = len(set(main_aua[(main_aua['ViewData.Status'] == 'UCB') & \
#                                                                    (main_aua['ViewData.Transaction Type'] == Transaction_type)]['Side_0_1_UniqueIds']))
#        
#        Transaction_Type_dict = update_dict_to_output_breakids_number_pct(fun_dict = Transaction_Type_dict, \
#                                                                          fun_aua_df = main_aua, \
#                                                                          fun_loop_transaction_type = Transaction_type, \
#                                                                          fun_count = count_closed_breaks_for_transaction_type, \
#                                                                          fun_Side_0_1_UniqueIds_list = Side_0_1_UniqueIds_for_Transaction_type)
            
        
        Side_0_1_UniqueIds_closed_all_list.extend(Side_0_1_UniqueIds_for_Transaction_type)
        print('\n' + Transaction_type + '\n')
#        pprint.pprint(dictionary_exclude_keys(fun_dict = Transaction_Type_dict.get(Transaction_type),                                      fun_keys_to_exclude = {'side','Transaction_Type','Side_meo_training_df'}),                      width = 4)
    
    sys.stdout = orig_stdout
    f.close()
    
#    count_all_closed_breaks = len(set(main_aua[(main_aua['ViewData.Status'] == 'UCB')]                                               ['Side_0_1_UniqueIds']))
    
#    aua_closed_dict = {'All_Closed_Breaks' : {}}
#    aua_closed_dict = update_dict_to_output_breakids_number_pct(fun_dict = aua_closed_dict,\
#                                                                fun_aua_df = main_aua, \
#                                                                fun_loop_transaction_type = 'All_Closed_Breaks', \
#                                                                fun_count = count_all_closed_breaks, \
#                                                                fun_Side_0_1_UniqueIds_list = Side_0_1_UniqueIds_closed_all_list)
    
#    write_dict_at_top(fun_filename = filepath_stdout, \
#                      fun_dict_to_add = aua_closed_dict)
    
    return(Side_0_1_UniqueIds_closed_all_list)

#### Closed break functions - End #### 

#### Break Prediction functions - Begin #### 

def equals_fun(a,b):
    if a == b:
        return 1
    else:
        return 0

vec_equals_fun = np.vectorize(equals_fun)


def descclean(com,cat_list):
    cat_all1 = []
    list1 = cat_list
    m = 0
    if (type(com) == str):
        com = com.lower()
        com1 =  re.split("[,/. \-!?:]+", com)
        
        
        
        for item in list1:
            if (type(item) == str):
                item = item.lower()
                item1 = item.split(' ')
                lst3 = [value for value in item1 if value in com1] 
                if len(lst3) == len(item1):
                    cat_all1.append(item)
                    m = m+1
            
                else:
                    m = m
            else:
                    m = 0
    else:
        m = 0
    

            
    if m >0 :
        return list(set(cat_all1))
    else:
        if ((type(com)==str)):
            if (len(com1)<4):
                if ((len(com1)==1) & com1[0].startswith('20')== True):
                    return 'swap id'
                else:
                    return com
            else:
                return 'NA'
        else:
            return 'NA'

def currcln(x):
    if (type(x)==list):
        return x
      
    else:
       
        
        if x == 'NA':
            return "NA"
        elif (('dollar' in x) | ('dollars' in x )):
            return 'dollar'
        elif (('pound' in x) | ('pounds' in x)):
            return 'pound'
        elif ('yen' in x):
            return 'yen'
        elif ('euro' in x) :
            return 'euro'
        else:
            return x

def catcln1(cat,df):
    ret = []
    if (type(cat)==list):
        
        if 'equity swap settlement' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'equity swap' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'swap settlement' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'swap unwind' in cat:
            ret.append('swap unwind')
        #return 'swap unwind'
   
    
    
    
        else:
        
       
            for item in cat:
            
                a = df[df['Pairing']==item]['replace'].values[0]
                if a not in ret:
                    ret.append(a)
        return list(set(ret))
      
    else:
        return cat

def desccat(x):
    if isinstance(x, list):
        
        if 'equity swap settlement' in x:
            return 'swap settlement'
        elif 'collateral transfer' in x:
            return 'collateral transfer'
        elif 'dividend' in x:
            return 'dividend'
        elif (('loan' in x) & ('option' in x)):
            return 'option loan'
        
        elif (('interest' in x) & ('corp' in x) ):
            return 'corp loan'
        elif (('interest' in x) & ('loan' in x) ):
            return 'interest'
        else:
            return x[0]
    else:
        return x

def new_pf_mapping(x):
    if x=='GSIL':
        return 'GS'
    elif x == 'CITIGM':
        return 'CITI'
    elif x == 'JPMNA':
        return 'JPM'
    else:
        return x

def mhreplaced(item):
    word1 = []
    word2 = []
    if (type(item) == str):
    
        for items in item.split(' '):
            if (type(items) == str):
                items = items.lower()
                if items.isdigit() == False:
                    word1.append(items)
        
            
                for c in word1:
                    if c.endswith('MH')==False:
                        word2.append(c)
    
                words = ' '.join(word2)
                return words
    else:
        return item
    

def fundmatch(item):
    items = item.lower()
    items = item.replace(' ','') 
    return items

def is_num(item):
    try:
        float(item)
        return True
    except ValueError:
        return False

def is_date_format(item):
    try:
        parse(item, fuzzy=False)
        return True
    
    except ValueError:
        return False
    
def date_edge_cases(item):
    if len(item) == 5 and item[2] =='/' and is_num(item[:2]) and is_num(item[3:]):
        return True
    return False

def nan_fun(x):
    if x=='nan':
        return 1
    else:
        return 0

def a_keymatch(a_cusip, a_isin):
    
    pb_nan = 0
    a_common_key = 'NA' 
    if a_cusip=='nan' and a_isin =='nan':
        pb_nan =1
    elif(a_cusip!='nan' and a_isin == 'nan'):
        a_common_key = a_cusip
    elif(a_cusip =='nan' and a_isin !='nan'):
        a_common_key = a_isin
    else:
        a_common_key = a_isin
        
    return (pb_nan, a_common_key)

def b_keymatch(b_cusip, b_isin):
    accounting_nan = 0
    b_common_key = 'NA'
    if b_cusip =='nan' and b_isin =='nan':
        accounting_nan =1
    elif (b_cusip!='nan' and b_isin == 'nan'):
        b_common_key = b_cusip
    elif(b_cusip =='nan' and b_isin !='nan'):
        b_common_key = b_isin
    else:
        b_common_key = b_isin
    return (accounting_nan, b_common_key)


def nan_equals_fun(a,b):
    if a==1 and b==1:
        return 1
    else:
        return 0


def new_key_match_fun(a,b,c):
    if a==b and c==0:
        return 1
    else:
        return 0

def  clean_text(df, text_field, new_text_field_name):
    df[text_field] = df[text_field].astype(str)
    df[new_text_field_name] = df[text_field].str.lower()
    
    
    
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: re.sub(r"\d+", "", x))
    df[new_text_field_name] = df[new_text_field_name].str.replace('usd','')
    df[new_text_field_name] = df[new_text_field_name].str.replace('eur0','')
    df[new_text_field_name] = df[new_text_field_name].str.replace(' usd','')
    df[new_text_field_name] = df[new_text_field_name].str.replace(' euro','')

    df[new_text_field_name] = df[new_text_field_name].str.replace(' eur','')
    df[new_text_field_name] = df[new_text_field_name].str.replace('eur','')
    
    return df

def umr_seg(X_test):
    b_count = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].value_counts().reset_index(name='count')
    b_unique = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].unique().reset_index()
    
    b_unique['len'] = b_unique['Predicted_action'].str.len()
    b_count2 = pd.merge(b_count, b_unique.drop('Predicted_action',1), on='SideB.ViewData.Side0_UniqueIds', how='left')
    umr_table = b_count2[(b_count2['Predicted_action']=='UMR_One_to_One') & (b_count2['count']==1) & (b_count2['len']<=2)]
    return umr_table['SideB.ViewData.Side0_UniqueIds'].values

def normalize_final_no_pair_table_col_names(fun_final_no_pair_table):
    final_no_pair_table_col_names_mapping_dict = {
                                      'SideA.ViewData.Side1_UniqueIds' : 'ViewData.Side1_UniqueIds',
                                      'SideB.ViewData.Side0_UniqueIds' : 'ViewData.Side0_UniqueIds',
                                      'SideA.ViewData.BreakID_A_side' : 'ViewData.BreakID_Side1', 
                                      'SideB.ViewData.BreakID_B_side' : 'ViewData.BreakID_Side0'
                                      }
    fun_final_no_pair_table.rename(columns = final_no_pair_table_col_names_mapping_dict, inplace = True)
    return(fun_final_no_pair_table)

def no_pair_seg(X_test):
    
    b_side_agg = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action_2'].unique().reset_index()
    a_side_agg = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action_2'].unique().reset_index()
    
    b_side_agg['len'] = b_side_agg['Predicted_action_2'].str.len()
    b_side_agg['No_Pair_flag'] = b_side_agg['Predicted_action_2'].apply(lambda x: 1 if 'No-Pair' in x else 0)

    a_side_agg['len'] = a_side_agg['Predicted_action_2'].str.len()
    a_side_agg['No_Pair_flag'] = a_side_agg['Predicted_action_2'].apply(lambda x: 1 if 'No-Pair' in x else 0)
    
    no_pair_ids_b_side = b_side_agg[(b_side_agg['len']==1) & (b_side_agg['No_Pair_flag']==1)]['SideB.ViewData.Side0_UniqueIds'].values

    no_pair_ids_a_side = a_side_agg[(a_side_agg['len']==1) & (a_side_agg['No_Pair_flag']==1)]['SideA.ViewData.Side1_UniqueIds'].values
    
    return no_pair_ids_b_side, no_pair_ids_a_side
 
def subSum(numbers,total):
    for length in range(1, 3):
        if len(numbers) < length or length < 1:
            return []
        for index,number in enumerate(numbers):
            if length == 1 and np.isclose(number, total,atol=0.25).any():
                return [number]
            subset = subSum(numbers[index+1:],total-number)
            if subset: 
                return [number] + subset
        return []

def one_to_one_umb(data):
    
    count = data['SideB.ViewData.Side0_UniqueIds'].value_counts().reset_index(name='count0')
    id0s = count[count['count0']==1]['index'].unique()
    id1s = data[data['SideB.ViewData.Side0_UniqueIds'].isin(id0s)]['SideA.ViewData.Side1_UniqueIds']
    
    count1 = data['SideA.ViewData.Side1_UniqueIds'].value_counts().reset_index(name='count1')
    final_ids = count1[(count1['count1']==1) & (count1['index'].isin(id1s))]['index'].unique()
    return final_ids

def no_pair_seg2(X_test):
    
    b_side_agg = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].unique().reset_index()
    a_side_agg = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action'].unique().reset_index()
    
    b_side_agg['len'] = b_side_agg['Predicted_action'].str.len()
    b_side_agg['No_Pair_flag'] = b_side_agg['Predicted_action'].apply(lambda x: 1 if 'No-Pair' in x else 0)

    a_side_agg['len'] = a_side_agg['Predicted_action'].str.len()
    a_side_agg['No_Pair_flag'] = a_side_agg['Predicted_action'].apply(lambda x: 1 if 'No-Pair' in x else 0)
    
    no_pair_ids_b_side = b_side_agg[(b_side_agg['len']==1) & (b_side_agg['No_Pair_flag']==1)]['SideB.ViewData.Side0_UniqueIds'].values

    no_pair_ids_a_side = a_side_agg[(a_side_agg['len']==1) & (a_side_agg['No_Pair_flag']==1)]['SideA.ViewData.Side1_UniqueIds'].values
    
    return no_pair_ids_b_side, no_pair_ids_a_side

def return_int_list(list_x):
    return [int(i) for i in list_x]
    
def normalize_bp_acct_col_names(fun_df):
    bp_acct_col_names_mapping_dict = {
                                      'ViewData.Cust Net Amount' : 'ViewData.B-P Net Amount',
                                      'ViewData.Cust Net Amount Difference' : 'ViewData.B-P Net Amount Difference',
                                      'ViewData.Cust Net Amount Difference Absolute' : 'ViewData.B-P Net Amount Difference Absolute',
                                      'ViewData.CP Net Amount' : 'ViewData.B-P Net Amount',
                                      'ViewData.CP Net Amount Difference' : 'ViewData.B-P Net Amount Difference',
                                      'ViewData.CP Net Amount Difference Absolute' : 'ViewData.B-P Net Amount Difference Absolute',
                                      'ViewData.PMSVendor Net Amount' : 'ViewData.Accounting Net Amount'
                                        }
    fun_df.rename(columns = bp_acct_col_names_mapping_dict, inplace = True)
    return(fun_df)

def find_BreakID_and_other_cols_in_meo_for_Side_0_1_UniqueIds_value(fun_string_value_of_Side_0_1_UniqueIds, fun_meo_df, fun_side, fun_other_cols_list = None):
    if fun_other_cols_list is None:
        all_cols_to_find = ['ViewData.BreakID']
    else:
        all_cols_to_find = fun_other_cols_list + ['ViewData.BreakID']
    if(fun_side == 0):
        return(fun_meo_df[fun_meo_df['ViewData.Side0_UniqueIds'] == fun_string_value_of_Side_0_1_UniqueIds][all_cols_to_find])
    elif(fun_side == 1):
        return(fun_meo_df[fun_meo_df['ViewData.Side1_UniqueIds'] == fun_string_value_of_Side_0_1_UniqueIds][all_cols_to_find])
    else:
        return 0

def contains_multiple_values_in_either_Side_0_or_1_UniqueIds_for_expected_single_sided_status(fun_row):
    
    if(',' in str(fun_row['ViewData.Side0_UniqueIds'])):
        Side_0_contains_comma = 1
    else:
        Side_0_contains_comma = 0

    if(',' in str(fun_row['ViewData.Side1_UniqueIds'])):
        Side_1_contains_comma = 1
    else:
        Side_1_contains_comma = 0
    
    if((str(fun_row['ViewData.Status']) in ['OB','SDB','UOB','CNF','CMF']) and ((Side_0_contains_comma == 1) or (Side_1_contains_comma == 1))):
        return('remove')
    else:
        return('keep')

def find_Side_0_1_UniqueIds_and_other_cols_in_meo_for_BreakID_value(fun_string_value_of_BreakID,fun_meo_df,fun_other_cols_list = None):
    if fun_other_cols_list is None:
        all_cols_to_find = ['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','ViewData.Status']
    else:
        all_cols_to_find = fun_other_cols_list + ['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','ViewData.Status']
    fun_meo_df['ViewData.BreakID'] = fun_meo_df['ViewData.BreakID'].astype(str)
    return(fun_meo_df[fun_meo_df['ViewData.BreakID'] == fun_string_value_of_BreakID][all_cols_to_find])

def make_Side0_Side1_columns_for_final_smb_ob_table_row_apply(row, fun_side):
#    print(row)

    if(fun_side == 0):
        if(row['Side0_UniqueIds_OB'] == ''):
            return(row['Side0_UniqueIds_SMB'])
        else:
            return(row['Side0_UniqueIds_OB'] + ',' + row['Side0_UniqueIds_SMB'])
    elif(fun_side == 1):
        if(row['Side1_UniqueIds_OB'] == ''):
            return(row['Side1_UniqueIds_SMB'])
        else:
            return(row['Side1_UniqueIds_OB'] + ',' + row['Side1_UniqueIds_SMB'])
    
def make_Side0_Side1_columns_for_final_smb_ob_table(fun_final_smb_ob_table, fun_meo_df):
    fun_final_smb_ob_table = pd.merge(fun_final_smb_ob_table,fun_meo_df[['ViewData.BreakID','ViewData.Side0_UniqueIds']], left_on = 'BreakID_OB', right_on = 'ViewData.BreakID')
    fun_final_smb_ob_table.drop('ViewData.BreakID', axis = 1, inplace = True)
    fun_final_smb_ob_table.rename(columns = {'ViewData.Side0_UniqueIds' : 'Side0_UniqueIds_OB'}, inplace = True) 

    fun_final_smb_ob_table = pd.merge(fun_final_smb_ob_table,fun_meo_df[['ViewData.BreakID','ViewData.Side1_UniqueIds']], left_on = 'BreakID_OB', right_on = 'ViewData.BreakID')
    fun_final_smb_ob_table.drop('ViewData.BreakID', axis = 1, inplace = True)
    fun_final_smb_ob_table.rename(columns = {'ViewData.Side1_UniqueIds' : 'Side1_UniqueIds_OB'}, inplace = True) 

    fun_final_smb_ob_table = pd.merge(fun_final_smb_ob_table,fun_meo_df[['ViewData.BreakID','ViewData.Side0_UniqueIds']], left_on = 'BreakID_SMB', right_on = 'ViewData.BreakID')
    fun_final_smb_ob_table.drop('ViewData.BreakID', axis = 1, inplace = True)
    fun_final_smb_ob_table.rename(columns = {'ViewData.Side0_UniqueIds' : 'Side0_UniqueIds_SMB'}, inplace = True) 

    fun_final_smb_ob_table = pd.merge(fun_final_smb_ob_table,fun_meo_df[['ViewData.BreakID','ViewData.Side1_UniqueIds']], left_on = 'BreakID_SMB', right_on = 'ViewData.BreakID')
    fun_final_smb_ob_table.drop('ViewData.BreakID', axis = 1, inplace = True)
    fun_final_smb_ob_table.rename(columns = {'ViewData.Side1_UniqueIds' : 'Side1_UniqueIds_SMB'}, inplace = True) 

    fun_final_smb_ob_table['Side0_UniqueIds_OB'] = fun_final_smb_ob_table['Side0_UniqueIds_OB'].astype(str)            
    fun_final_smb_ob_table['Side1_UniqueIds_OB'] = fun_final_smb_ob_table['Side1_UniqueIds_OB'].astype(str)            
    fun_final_smb_ob_table['Side0_UniqueIds_SMB'] = fun_final_smb_ob_table['Side0_UniqueIds_SMB'].astype(str)            
    fun_final_smb_ob_table['Side1_UniqueIds_SMB'] = fun_final_smb_ob_table['Side1_UniqueIds_SMB'].astype(str)            

    fun_final_smb_ob_table['Side0_UniqueIds_OB'] = fun_final_smb_ob_table['Side0_UniqueIds_OB'].replace('None','')            
    fun_final_smb_ob_table['Side1_UniqueIds_OB'] = fun_final_smb_ob_table['Side1_UniqueIds_OB'].replace('None','')            
    fun_final_smb_ob_table['Side0_UniqueIds_SMB'] = fun_final_smb_ob_table['Side0_UniqueIds_SMB'].replace('None','')            
    fun_final_smb_ob_table['Side1_UniqueIds_SMB'] = fun_final_smb_ob_table['Side1_UniqueIds_SMB'].replace('None','')            

    fun_final_smb_ob_table['Side0_UniqueIds_OB'] = fun_final_smb_ob_table['Side0_UniqueIds_OB'].replace('nan','')            
    fun_final_smb_ob_table['Side1_UniqueIds_OB'] = fun_final_smb_ob_table['Side1_UniqueIds_OB'].replace('nan','')
    fun_final_smb_ob_table['Side0_UniqueIds_SMB'] = fun_final_smb_ob_table['Side0_UniqueIds_SMB'].replace('nan','') 
    fun_final_smb_ob_table['Side1_UniqueIds_SMB'] = fun_final_smb_ob_table['Side1_UniqueIds_SMB'].replace('nan','')

    fun_final_smb_ob_table['Side0_UniqueIds'] = fun_final_smb_ob_table.apply(lambda row : make_Side0_Side1_columns_for_final_smb_ob_table_row_apply(row, fun_side = 0),axis = 1,result_type="expand")
    fun_final_smb_ob_table['Side1_UniqueIds'] = fun_final_smb_ob_table.apply(lambda row : make_Side0_Side1_columns_for_final_smb_ob_table_row_apply(row, fun_side = 1),axis = 1,result_type="expand")
#    fun_final_smb_ob_table.iloc[fun_final_smb_ob_table['Side0_UniqueIds_OB'] == '', 'Side0_UniqueIds'] = fun_final_smb_ob_table['Side0_UniqueIds_SMB']
#    fun_final_smb_ob_table.iloc[fun_final_smb_ob_table['Side0_UniqueIds_OB'] != '', 'Side0_UniqueIds'] = fun_final_smb_ob_table['Side0_UniqueIds_OB'] + fun_final_smb_ob_table['Side0_UniqueIds_SMB']
#    fun_final_smb_ob_table.iloc[fun_final_smb_ob_table['Side1_UniqueIds_OB'] == '', 'Side1_UniqueIds'] = fun_final_smb_ob_table['Side1_UniqueIds_SMB']
#    fun_final_smb_ob_table.iloc[fun_final_smb_ob_table['Side1_UniqueIds_OB'] != '', 'Side1_UniqueIds'] = fun_final_smb_ob_table['Side1_UniqueIds_OB'] + fun_final_smb_ob_table['Side1_UniqueIds_SMB']

    fun_final_smb_ob_table.drop(['Side0_UniqueIds_OB','Side1_UniqueIds_OB','Side0_UniqueIds_SMB','Side1_UniqueIds_SMB'], axis = 1, inplace = True)

    return(fun_final_smb_ob_table)


date_numbers_list = [16]
                     #2,3,4,
                    # 7,8,9,10,11,
                    # 14,15,16,17,18,
                    # 21,22,23,24,25,
                    # 28,29,30]

client = 'OakTree'

setup_code = '379'

today = date.today()
d1 = datetime.strptime(today.strftime("%Y-%m-%d"),"%Y-%m-%d")
desired_date = d1 - timedelta(days=4)
desired_date_str = desired_date.strftime("%Y-%m-%d")
date_input = desired_date_str

#filepaths_AUA = '//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/' + client + '/JuneData/AUA/AUACollections.AUA_HST_RecData_' + setup_code + '_' + str(date_input) + '.csv'
#filepaths_MEO = '//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/' + client + '/JuneData/MEO/MeoCollections.MEO_HST_RecData_' + setup_code + '_' + str(date_input) + '.csv'
filepaths_no_pair_id_data = '//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/' + client + '/UAT_Run/X_Test_' + setup_code + '/no_pair_ids_' + setup_code + '_' + str(date_input) + '.csv'
filepaths_no_pair_id_no_data_warning = '//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/' + client + '/UAT_Run/X_Test_' + setup_code + '/WARNING_no_pair_ids_' + setup_code + str(date_input) + '.csv'


mngdb_obj_1_for_reading_and_writing_in_uat_server = mngdb(param_without_ssh  = True, param_without_RabbitMQ_pipeline = True,
                 param_SSH_HOST = None, param_SSH_PORT = None,
                 param_SSH_USERNAME = None, param_SSH_PASSWORD = None,
                 param_MONGO_HOST = '10.1.15.137', param_MONGO_PORT = 27017,
                 param_MONGO_USERNAME = 'mongouseradmin', param_MONGO_PASSWORD = '@L0ck&Key')
mngdb_obj_1_for_reading_and_writing_in_uat_server.connect_with_or_without_ssh()
db_1_for_MEO_data = mngdb_obj_1_for_reading_and_writing_in_uat_server.client['ReconDB_ML']


query_1_for_MEO_data = db_1_for_MEO_data['RecData_' + setup_code].find({ 
                                                                     "LastPerformedAction": 31
                                                             },
                                                             {
                                                                     "DataSides" : 1,
                                                                     "BreakID" : 1,
                                                                     "LastPerformedAction" : 1,
                                                                     "TaskInstanceID" : 1,
                                                                     "SourceCombinationCode" : 1,
                                                                     "MetaData" : 1, 
                                                                     "ViewData" : 1
                                                             })
list_of_dicts_query_result_1 = list(query_1_for_MEO_data)

meo_df = json_normalize(list_of_dicts_query_result_1)
meo_df = meo_df.loc[:,meo_df.columns.str.startswith('ViewData')]
meo_df['ViewData.Task Business Date'] = meo_df['ViewData.Task Business Date'].apply(dt.datetime.isoformat) 
print(meo_df.shape[0])
meo_df.drop_duplicates(keep=False, inplace = True)
meo_df = normalize_bp_acct_col_names(fun_df = meo_df)

#Change added on 17-12-2020 to remove records with multiple values of Side0 and Side1 UniqueIds for statuses like OB,UOB,SDB,CNF and CMF. Typically, these statuses should have single values in Side0 and Side1 UniqueIds. So records not following expected behviour are removed

meo_df['remove_or_keep_for_multiple_uniqueids_in_ob_issue'] = meo_df.apply(lambda row : contains_multiple_values_in_either_Side_0_or_1_UniqueIds_for_expected_single_sided_status(fun_row = row), axis = 1,result_type="expand")
meo_df = meo_df[~(meo_df['remove_or_keep_for_multiple_uniqueids_in_ob_issue'] == 'remove')]

meo = meo_df[new_cols]
print('meo size')
print(meo.shape[0])



ModuleNotFoundError: No module named 'ViteosMongoDB'

In [165]:
umb_carry_forward_df = meo_df[meo_df['ViewData.Status'] == 'UMB']

meo_df_taskids = list(meo_df['ViewData.Task ID'].unique())


In [2]:
import numpy as np
import pandas as pd
#from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from tqdm import tqdm
import pickle
import datetime as dt
import sys
#from ViteosMongoDB import  ViteosMongoDB_Class as mngdb
from datetime import datetime,date,timedelta
from pandas.io.json import json_normalize
import dateutil.parser
from difflib import SequenceMatcher
import pprint
import json
from pandas import merge
import re

import dask.dataframe as dd
import glob
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from dateutil.parser import parse
import operator
import itertools
from sklearn.feature_extraction.text import CountVectorizer

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

from fuzzywuzzy import fuzz
import random
import decimal

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
pwd

'C:\\Users\\consultant136\\ML1.0'

In [130]:
import numpy as np
import pandas as pd
meo = pd.read_csv('meo_df_setup_379_date_2020-12-08.csv')

In [131]:
common_cols = ['ViewData.Accounting Net Amount', 'ViewData.Age',
'ViewData.Age WK', 'ViewData.Asset Type Category',
'ViewData.B-P Net Amount', 'ViewData.Base Net Amount','ViewData.CUSIP', 
 'ViewData.Cancel Amount',
       'ViewData.Cancel Flag',
#'ViewData.Commission',
        'ViewData.Currency', 'ViewData.Custodian',
       'ViewData.Custodian Account',
       'ViewData.Description','ViewData.Department', 'ViewData.ExpiryDate', 'ViewData.Fund',
       'ViewData.ISIN',
       'ViewData.Investment Type',
      # 'ViewData.Keys',
       'ViewData.Mapped Custodian Account',
       'ViewData.Net Amount Difference',
       'ViewData.Net Amount Difference Absolute',
        #'ViewData.OTE Ticker',
        'ViewData.Price',
       'ViewData.Prime Broker', 'ViewData.Quantity',
       'ViewData.SEDOL', 'ViewData.SPM ID', 'ViewData.Settle Date',
       
  #  'ViewData.Strike Price',
               'Date',
       'ViewData.Ticker', 'ViewData.Trade Date',
       'ViewData.Transaction Category',
       'ViewData.Transaction Type', 'ViewData.Underlying Cusip',
       'ViewData.Underlying ISIN',
       'ViewData.Underlying Sedol','filter_key','ViewData.Status','ViewData.BreakID',
              'ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','ViewData._ID']

model_cols = [
            'SideA.ViewData.B-P Net Amount', 
              #'SideA.ViewData.Cancel Flag', 
              #'SideA.new_desc_cat',
             # 'SideA.ViewData.Description',
             # 'SideA.ViewData.Department',
   
    
              
             # 'SideA.ViewData.Price',
             # 'SideA.ViewData.Quantity',
             #'SideA.ViewData.Investment Type', 
              #'SideA.ViewData.Asset Type Category', 
              'SideB.ViewData.Accounting Net Amount', 
              #'SideB.ViewData.Cancel Flag', 
             # 'SideB.ViewData.Description',
              # 'SideB.ViewData.Department',
              
             # 'SideB.ViewData.Price',
             # 'SideB.ViewData.Quantity',
             # 'SideB.new_desc_cat',
             # 'SideB.ViewData.Investment Type', 
              #'SideB.ViewData.Asset Type Category', 
              'Trade_Date_match', 'Settle_Date_match', 
                'Amount_diff_2', 
              'Trade_date_diff', 'Settle_date_diff', 'SideA.ISIN_NA', 'SideB.ISIN_NA', 
             # 'ViewData.Combined Fund',
              'ViewData.Combined Transaction Type', 'Combined_Desc','Combined_TType',
             # 'SideA.TType', 'SideB.TType', 
              'abs_amount_flag',
    'tt_map_flag', 
              'All_key_nan','new_key_match', 'new_pb1',
              'SideB.Date','SideA.ViewData.Settle Date','SideB.ViewData.Settle Date',
            'SideA.ViewData._ID', 'SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds',
              'SideB.ViewData.Status', 'SideB.ViewData.BreakID_B_side',
              'SideA.ViewData.Status', 'SideA.ViewData.BreakID_A_side'] 
              #'label']

model_cols_2 =[#'SideA.ViewData.B-P Net Amount', 
              #'SideA.ViewData.Cancel Flag', 
              #'SideA.new_desc_cat',
             # 'SideA.ViewData.Description',
             # 'SideA.ViewData.Department',
   
    
              
             # 'SideA.ViewData.Price',
             # 'SideA.ViewData.Quantity',
             #'SideA.ViewData.Investment Type', 
              #'SideA.ViewData.Asset Type Category', 
              #'SideB.ViewData.Accounting Net Amount', 
              #'SideB.ViewData.Cancel Flag', 
             # 'SideB.ViewData.Description',
              # 'SideB.ViewData.Department',
              
             # 'SideB.ViewData.Price',
             # 'SideB.ViewData.Quantity',
             # 'SideB.new_desc_cat',
             # 'SideB.ViewData.Investment Type', 
              #'SideB.ViewData.Asset Type Category', 
              'Trade_Date_match', 'Settle_Date_match', 
              #  'Amount_diff_2', 
              'Trade_date_diff', 'Settle_date_diff', 'SideA.ISIN_NA', 'SideB.ISIN_NA', 
             # 'ViewData.Combined Fund',
              'ViewData.Combined Transaction Type', 'Combined_Desc','Combined_TType',
             # 'SideA.TType', 'SideB.TType', 
              'abs_amount_flag',
    'tt_map_flag', 
              'All_key_nan','new_key_match', 'new_pb1',
              'SideB.Date','SideA.ViewData.Settle Date','SideB.ViewData.Settle Date',
            'SideA.ViewData._ID', 'SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds',
              'SideB.ViewData.Status', 'SideB.ViewData.BreakID_B_side',
              'SideA.ViewData.Status', 'SideA.ViewData.BreakID_A_side'] 
              #'label']


In [132]:
#### Break Prediction functions - Begin #### 

def equals_fun(a,b):
    if a == b:
        return 1
    else:
        return 0

vec_equals_fun = np.vectorize(equals_fun)


def descclean(com,cat_list):
    cat_all1 = []
    list1 = cat_list
    m = 0
    if (type(com) == str):
        com = com.lower()
        com1 =  re.split("[,/. \-!?:]+", com)
        
        
        
        for item in list1:
            if (type(item) == str):
                item = item.lower()
                item1 = item.split(' ')
                lst3 = [value for value in item1 if value in com1] 
                if len(lst3) == len(item1):
                    cat_all1.append(item)
                    m = m+1
            
                else:
                    m = m
            else:
                    m = 0
    else:
        m = 0
    

            
    if m >0 :
        return list(set(cat_all1))
    else:
        if ((type(com)==str)):
            if (len(com1)<4):
                if ((len(com1)==1) & com1[0].startswith('20')== True):
                    return 'swap id'
                else:
                    return com
            else:
                return 'NA'
        else:
            return 'NA'

def currcln(x):
    if (type(x)==list):
        return x
      
    else:
       
        
        if x == 'NA':
            return "NA"
        elif (('dollar' in x) | ('dollars' in x )):
            return 'dollar'
        elif (('pound' in x) | ('pounds' in x)):
            return 'pound'
        elif ('yen' in x):
            return 'yen'
        elif ('euro' in x) :
            return 'euro'
        else:
            return x

def catcln1(cat,df):
    ret = []
    if (type(cat)==list):
        
        if 'equity swap settlement' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'equity swap' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'swap settlement' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'swap unwind' in cat:
            ret.append('swap unwind')
        #return 'swap unwind'
   
    
    
    
        else:
        
       
            for item in cat:
            
                a = df[df['Pairing']==item]['replace'].values[0]
                if a not in ret:
                    ret.append(a)
        return list(set(ret))
      
    else:
        return cat

def desccat(x):
    if isinstance(x, list):
        
        if 'equity swap settlement' in x:
            return 'swap settlement'
        elif 'collateral transfer' in x:
            return 'collateral transfer'
        elif 'dividend' in x:
            return 'dividend'
        elif (('loan' in x) & ('option' in x)):
            return 'option loan'
        
        elif (('interest' in x) & ('corp' in x) ):
            return 'corp loan'
        elif (('interest' in x) & ('loan' in x) ):
            return 'interest'
        else:
            return x[0]
    else:
        return x

def new_pf_mapping(x):
    if x=='GSIL':
        return 'GS'
    elif x == 'CITIGM':
        return 'CITI'
    elif x == 'JPMNA':
        return 'JPM'
    else:
        return x

def mhreplaced(item):
    word1 = []
    word2 = []
    if (type(item) == str):
    
        for items in item.split(' '):
            if (type(items) == str):
                items = items.lower()
                if items.isdigit() == False:
                    word1.append(items)
        
            
                for c in word1:
                    if c.endswith('MH')==False:
                        word2.append(c)
    
                words = ' '.join(word2)
                return words
    else:
        return item
    

def fundmatch(item):
    items = item.lower()
    items = item.replace(' ','') 
    return items

def is_num(item):
    try:
        float(item)
        return True
    except ValueError:
        return False

def is_date_format(item):
    try:
        parse(item, fuzzy=False)
        return True
    
    except ValueError:
        return False
    
def date_edge_cases(item):
    if len(item) == 5 and item[2] =='/' and is_num(item[:2]) and is_num(item[3:]):
        return True
    return False

def nan_fun(x):
    if x=='nan':
        return 1
    else:
        return 0

def a_keymatch(a_cusip, a_isin):
    
    pb_nan = 0
    a_common_key = 'NA' 
    if a_cusip=='nan' and a_isin =='nan':
        pb_nan =1
    elif(a_cusip!='nan' and a_isin == 'nan'):
        a_common_key = a_cusip
    elif(a_cusip =='nan' and a_isin !='nan'):
        a_common_key = a_isin
    else:
        a_common_key = a_isin
        
    return (pb_nan, a_common_key)

def b_keymatch(b_cusip, b_isin):
    accounting_nan = 0
    b_common_key = 'NA'
    if b_cusip =='nan' and b_isin =='nan':
        accounting_nan =1
    elif (b_cusip!='nan' and b_isin == 'nan'):
        b_common_key = b_cusip
    elif(b_cusip =='nan' and b_isin !='nan'):
        b_common_key = b_isin
    else:
        b_common_key = b_isin
    return (accounting_nan, b_common_key)


def nan_equals_fun(a,b):
    if a==1 and b==1:
        return 1
    else:
        return 0


def new_key_match_fun(a,b,c):
    if a==b and c==0:
        return 1
    else:
        return 0

def  clean_text(df, text_field, new_text_field_name):
    df[text_field] = df[text_field].astype(str)
    df[new_text_field_name] = df[text_field].str.lower()
    
    
    
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: re.sub(r"\d+", "", x))
    df[new_text_field_name] = df[new_text_field_name].str.replace('usd','')
    df[new_text_field_name] = df[new_text_field_name].str.replace('eur0','')
    df[new_text_field_name] = df[new_text_field_name].str.replace(' usd','')
    df[new_text_field_name] = df[new_text_field_name].str.replace(' euro','')

    df[new_text_field_name] = df[new_text_field_name].str.replace(' eur','')
    df[new_text_field_name] = df[new_text_field_name].str.replace('eur','')
    
    return df

def umr_seg(X_test):
    b_count = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].value_counts().reset_index(name='count')
    b_unique = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].unique().reset_index()
    
    b_unique['len'] = b_unique['Predicted_action'].str.len()
    b_count2 = pd.merge(b_count, b_unique.drop('Predicted_action',1), on='SideB.ViewData.Side0_UniqueIds', how='left')
    umr_table = b_count2[(b_count2['Predicted_action']=='UMR_One_to_One') & (b_count2['count']==1) & (b_count2['len']<=2)]
    return umr_table['SideB.ViewData.Side0_UniqueIds'].values

def normalize_final_no_pair_table_col_names(fun_final_no_pair_table):
    final_no_pair_table_col_names_mapping_dict = {
                                      'SideA.ViewData.Side1_UniqueIds' : 'ViewData.Side1_UniqueIds',
                                      'SideB.ViewData.Side0_UniqueIds' : 'ViewData.Side0_UniqueIds',
                                      'SideA.ViewData.BreakID_A_side' : 'ViewData.BreakID_Side1', 
                                      'SideB.ViewData.BreakID_B_side' : 'ViewData.BreakID_Side0'
                                      }
    fun_final_no_pair_table.rename(columns = final_no_pair_table_col_names_mapping_dict, inplace = True)
    return(fun_final_no_pair_table)

def no_pair_seg(X_test):
    
    b_side_agg = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action_2'].unique().reset_index()
    a_side_agg = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action_2'].unique().reset_index()
    
    b_side_agg['len'] = b_side_agg['Predicted_action_2'].str.len()
    b_side_agg['No_Pair_flag'] = b_side_agg['Predicted_action_2'].apply(lambda x: 1 if 'No-Pair' in x else 0)

    a_side_agg['len'] = a_side_agg['Predicted_action_2'].str.len()
    a_side_agg['No_Pair_flag'] = a_side_agg['Predicted_action_2'].apply(lambda x: 1 if 'No-Pair' in x else 0)
    
    no_pair_ids_b_side = b_side_agg[(b_side_agg['len']==1) & (b_side_agg['No_Pair_flag']==1)]['SideB.ViewData.Side0_UniqueIds'].values

    no_pair_ids_a_side = a_side_agg[(a_side_agg['len']==1) & (a_side_agg['No_Pair_flag']==1)]['SideA.ViewData.Side1_UniqueIds'].values
    
    return no_pair_ids_b_side, no_pair_ids_a_side
 
def subSum(numbers,total):
    for length in range(1, 3):
        if len(numbers) < length or length < 1:
            return []
        for index,number in enumerate(numbers):
            if length == 1 and np.isclose(number, total,atol=0.25).any():
                return [number]
            subset = subSum(numbers[index+1:],total-number)
            if subset: 
                return [number] + subset
        return []

def one_to_one_umb(data):
    
    count = data['SideB.ViewData.Side0_UniqueIds'].value_counts().reset_index(name='count0')
    id0s = count[count['count0']==1]['index'].unique()
    id1s = data[data['SideB.ViewData.Side0_UniqueIds'].isin(id0s)]['SideA.ViewData.Side1_UniqueIds']
    
    count1 = data['SideA.ViewData.Side1_UniqueIds'].value_counts().reset_index(name='count1')
    final_ids = count1[(count1['count1']==1) & (count1['index'].isin(id1s))]['index'].unique()
    return final_ids

def no_pair_seg2(X_test):
    
    b_side_agg = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].unique().reset_index()
    a_side_agg = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action'].unique().reset_index()
    
    b_side_agg['len'] = b_side_agg['Predicted_action'].str.len()
    b_side_agg['No_Pair_flag'] = b_side_agg['Predicted_action'].apply(lambda x: 1 if 'No-Pair' in x else 0)

    a_side_agg['len'] = a_side_agg['Predicted_action'].str.len()
    a_side_agg['No_Pair_flag'] = a_side_agg['Predicted_action'].apply(lambda x: 1 if 'No-Pair' in x else 0)
    
    no_pair_ids_b_side = b_side_agg[(b_side_agg['len']==1) & (b_side_agg['No_Pair_flag']==1)]['SideB.ViewData.Side0_UniqueIds'].values

    no_pair_ids_a_side = a_side_agg[(a_side_agg['len']==1) & (a_side_agg['No_Pair_flag']==1)]['SideA.ViewData.Side1_UniqueIds'].values
    
    return no_pair_ids_b_side, no_pair_ids_a_side

def return_int_list(list_x):
    return [int(i) for i in list_x]
    
def normalize_bp_acct_col_names(fun_df):
    bp_acct_col_names_mapping_dict = {
                                      'ViewData.Cust Net Amount' : 'ViewData.B-P Net Amount',
                                      'ViewData.Cust Net Amount Difference' : 'ViewData.B-P Net Amount Difference',
                                      'ViewData.Cust Net Amount Difference Absolute' : 'ViewData.B-P Net Amount Difference Absolute',
                                      'ViewData.CP Net Amount' : 'ViewData.B-P Net Amount',
                                      'ViewData.CP Net Amount Difference' : 'ViewData.B-P Net Amount Difference',
                                      'ViewData.CP Net Amount Difference Absolute' : 'ViewData.B-P Net Amount Difference Absolute',
                                      'ViewData.PMSVendor Net Amount' : 'ViewData.Accounting Net Amount'
                                        }
    fun_df.rename(columns = bp_acct_col_names_mapping_dict, inplace = True)
    return(fun_df)

def find_BreakID_and_other_cols_in_meo_for_Side_0_1_UniqueIds_value(fun_string_value_of_Side_0_1_UniqueIds, fun_meo_df, fun_side, fun_other_cols_list = None):
    if fun_other_cols_list is None:
        all_cols_to_find = ['ViewData.BreakID']
    else:
        all_cols_to_find = fun_other_cols_list + ['ViewData.BreakID']
    if(fun_side == 0):
        return(fun_meo_df[fun_meo_df['ViewData.Side0_UniqueIds'] == fun_string_value_of_Side_0_1_UniqueIds][all_cols_to_find])
    elif(fun_side == 1):
        return(fun_meo_df[fun_meo_df['ViewData.Side1_UniqueIds'] == fun_string_value_of_Side_0_1_UniqueIds][all_cols_to_find])
    else:
        return 0

def contains_multiple_values_in_either_Side_0_or_1_UniqueIds_for_expected_single_sided_status(fun_row):
    
    if(',' in str(fun_row['ViewData.Side0_UniqueIds'])):
        Side_0_contains_comma = 1
    else:
        Side_0_contains_comma = 0

    if(',' in str(fun_row['ViewData.Side1_UniqueIds'])):
        Side_1_contains_comma = 1
    else:
        Side_1_contains_comma = 0
    
    if((str(fun_row['ViewData.Status']) in ['OB','SDB','UOB','CNF','CMF']) and ((Side_0_contains_comma == 1) or (Side_1_contains_comma == 1))):
        return('remove')
    else:
        return('keep')

def find_Side_0_1_UniqueIds_and_other_cols_in_meo_for_BreakID_value(fun_string_value_of_BreakID,fun_meo_df,fun_other_cols_list = None):
    if fun_other_cols_list is None:
        all_cols_to_find = ['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','ViewData.Status']
    else:
        all_cols_to_find = fun_other_cols_list + ['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','ViewData.Status']
    fun_meo_df['ViewData.BreakID'] = fun_meo_df['ViewData.BreakID'].astype(str)
    return(fun_meo_df[fun_meo_df['ViewData.BreakID'] == fun_string_value_of_BreakID][all_cols_to_find])

def make_Side0_Side1_columns_for_final_smb_ob_table_row_apply(row, fun_side):
#    print(row)

    if(fun_side == 0):
        if(row['Side0_UniqueIds_OB'] == ''):
            return(row['Side0_UniqueIds_SMB'])
        else:
            return(row['Side0_UniqueIds_OB'] + ',' + row['Side0_UniqueIds_SMB'])
    elif(fun_side == 1):
        if(row['Side1_UniqueIds_OB'] == ''):
            return(row['Side1_UniqueIds_SMB'])
        else:
            return(row['Side1_UniqueIds_OB'] + ',' + row['Side1_UniqueIds_SMB'])
    
def make_Side0_Side1_columns_for_final_smb_ob_table(fun_final_smb_ob_table, fun_meo_df):
    fun_final_smb_ob_table = pd.merge(fun_final_smb_ob_table,fun_meo_df[['ViewData.BreakID','ViewData.Side0_UniqueIds']], left_on = 'BreakID_OB', right_on = 'ViewData.BreakID')
    fun_final_smb_ob_table.drop('ViewData.BreakID', axis = 1, inplace = True)
    fun_final_smb_ob_table.rename(columns = {'ViewData.Side0_UniqueIds' : 'Side0_UniqueIds_OB'}, inplace = True) 

    fun_final_smb_ob_table = pd.merge(fun_final_smb_ob_table,fun_meo_df[['ViewData.BreakID','ViewData.Side1_UniqueIds']], left_on = 'BreakID_OB', right_on = 'ViewData.BreakID')
    fun_final_smb_ob_table.drop('ViewData.BreakID', axis = 1, inplace = True)
    fun_final_smb_ob_table.rename(columns = {'ViewData.Side1_UniqueIds' : 'Side1_UniqueIds_OB'}, inplace = True) 

    fun_final_smb_ob_table = pd.merge(fun_final_smb_ob_table,fun_meo_df[['ViewData.BreakID','ViewData.Side0_UniqueIds']], left_on = 'BreakID_SMB', right_on = 'ViewData.BreakID')
    fun_final_smb_ob_table.drop('ViewData.BreakID', axis = 1, inplace = True)
    fun_final_smb_ob_table.rename(columns = {'ViewData.Side0_UniqueIds' : 'Side0_UniqueIds_SMB'}, inplace = True) 

    fun_final_smb_ob_table = pd.merge(fun_final_smb_ob_table,fun_meo_df[['ViewData.BreakID','ViewData.Side1_UniqueIds']], left_on = 'BreakID_SMB', right_on = 'ViewData.BreakID')
    fun_final_smb_ob_table.drop('ViewData.BreakID', axis = 1, inplace = True)
    fun_final_smb_ob_table.rename(columns = {'ViewData.Side1_UniqueIds' : 'Side1_UniqueIds_SMB'}, inplace = True) 

    fun_final_smb_ob_table['Side0_UniqueIds_OB'] = fun_final_smb_ob_table['Side0_UniqueIds_OB'].astype(str)            
    fun_final_smb_ob_table['Side1_UniqueIds_OB'] = fun_final_smb_ob_table['Side1_UniqueIds_OB'].astype(str)            
    fun_final_smb_ob_table['Side0_UniqueIds_SMB'] = fun_final_smb_ob_table['Side0_UniqueIds_SMB'].astype(str)            
    fun_final_smb_ob_table['Side1_UniqueIds_SMB'] = fun_final_smb_ob_table['Side1_UniqueIds_SMB'].astype(str)            

    fun_final_smb_ob_table['Side0_UniqueIds_OB'] = fun_final_smb_ob_table['Side0_UniqueIds_OB'].replace('None','')            
    fun_final_smb_ob_table['Side1_UniqueIds_OB'] = fun_final_smb_ob_table['Side1_UniqueIds_OB'].replace('None','')            
    fun_final_smb_ob_table['Side0_UniqueIds_SMB'] = fun_final_smb_ob_table['Side0_UniqueIds_SMB'].replace('None','')            
    fun_final_smb_ob_table['Side1_UniqueIds_SMB'] = fun_final_smb_ob_table['Side1_UniqueIds_SMB'].replace('None','')            

    fun_final_smb_ob_table['Side0_UniqueIds_OB'] = fun_final_smb_ob_table['Side0_UniqueIds_OB'].replace('nan','')            
    fun_final_smb_ob_table['Side1_UniqueIds_OB'] = fun_final_smb_ob_table['Side1_UniqueIds_OB'].replace('nan','')
    fun_final_smb_ob_table['Side0_UniqueIds_SMB'] = fun_final_smb_ob_table['Side0_UniqueIds_SMB'].replace('nan','') 
    fun_final_smb_ob_table['Side1_UniqueIds_SMB'] = fun_final_smb_ob_table['Side1_UniqueIds_SMB'].replace('nan','')

    fun_final_smb_ob_table['Side0_UniqueIds'] = fun_final_smb_ob_table.apply(lambda row : make_Side0_Side1_columns_for_final_smb_ob_table_row_apply(row, fun_side = 0),axis = 1,result_type="expand")
    fun_final_smb_ob_table['Side1_UniqueIds'] = fun_final_smb_ob_table.apply(lambda row : make_Side0_Side1_columns_for_final_smb_ob_table_row_apply(row, fun_side = 1),axis = 1,result_type="expand")
#    fun_final_smb_ob_table.iloc[fun_final_smb_ob_table['Side0_UniqueIds_OB'] == '', 'Side0_UniqueIds'] = fun_final_smb_ob_table['Side0_UniqueIds_SMB']
#    fun_final_smb_ob_table.iloc[fun_final_smb_ob_table['Side0_UniqueIds_OB'] != '', 'Side0_UniqueIds'] = fun_final_smb_ob_table['Side0_UniqueIds_OB'] + fun_final_smb_ob_table['Side0_UniqueIds_SMB']
#    fun_final_smb_ob_table.iloc[fun_final_smb_ob_table['Side1_UniqueIds_OB'] == '', 'Side1_UniqueIds'] = fun_final_smb_ob_table['Side1_UniqueIds_SMB']
#    fun_final_smb_ob_table.iloc[fun_final_smb_ob_table['Side1_UniqueIds_OB'] != '', 'Side1_UniqueIds'] = fun_final_smb_ob_table['Side1_UniqueIds_OB'] + fun_final_smb_ob_table['Side1_UniqueIds_SMB']

    fun_final_smb_ob_table.drop(['Side0_UniqueIds_OB','Side1_UniqueIds_OB','Side0_UniqueIds_SMB','Side1_UniqueIds_SMB'], axis = 1, inplace = True)

    return(fun_final_smb_ob_table)

In [133]:
#Change made on 12-12-2020 as per Pratik to catch instances where a single SMB pairs off with a single OB. BreakIDs caught in this code piece will be removed from propogating down further. Also, these BreakIDs will be given the status of UMR with Predicted_action of UMR_One-Many_to_Many-One
#Begin change code made on 12-12-2020
meo2 = meo[meo['ViewData.Status'].isin(['OB','SMB','SPM','UMB'])]
meo2 = meo2.reset_index().drop('index',1)

meo2['ViewData.Net Amount Difference Absolute'] = np.round(meo2['ViewData.Net Amount Difference Absolute'],2)

abs_amount_count = meo2['ViewData.Net Amount Difference Absolute'].value_counts().reset_index()

duplicate_amount = abs_amount_count[abs_amount_count['ViewData.Net Amount Difference Absolute']==2]
duplicate_amount.columns = ['ViewData.Net Amount Difference Absolute','count']
duplicate_amount = duplicate_amount.reset_index().drop('index',1)

if duplicate_amount.shape[0]>0:
    meo3 = meo2[meo2['ViewData.Net Amount Difference Absolute'].isin(duplicate_amount['ViewData.Net Amount Difference Absolute'].unique())]
    meo3 = meo3.reset_index().drop('index',1)
    meo3 = meo3.sort_values(by='ViewData.Net Amount Difference Absolute')
    meo3 = meo3.reset_index().drop('index',1)
    
    smb_amount = meo3[meo3['ViewData.Status'].isin(['SMB'])]['ViewData.Net Amount Difference Absolute'].unique()
    umb_amount = meo3[meo3['ViewData.Status'].isin(['UMB'])]['ViewData.Net Amount Difference Absolute'].unique()
    
    smb_ob_table = meo3[meo3['ViewData.Net Amount Difference Absolute'].isin(smb_amount)]
    umb_ob_table = meo3[meo3['ViewData.Net Amount Difference Absolute'].isin(umb_amount)]
    
    ob_breakid = []
    smb_breakid = []
    for amount in smb_amount:
        ob = smb_ob_table[(smb_ob_table['ViewData.Net Amount Difference Absolute']==amount) & (smb_ob_table['ViewData.Status']=='OB')]
        smb = smb_ob_table[(smb_ob_table['ViewData.Net Amount Difference Absolute']==amount) & (smb_ob_table['ViewData.Status']=='SMB')]
#         if((ob.shape[0]==1) and (smb.shape[0]==1) and (ob['ViewData.Mapped Custodian Account'] == smb['ViewData.Mapped Custodian Account']) and (ob['ViewData.Currency'] == smb['ViewData.Currency']) and (ob['ViewData.Source Combination Code'] == smb['ViewData.Source Combination Code'])):

        if ob.shape[0]==1 and smb.shape[0]==1 :
#Change added on 17-12-2020 by Rohit to include filter on ob and smb. Below if statement is commented out and new if statement is included
            if((ob['ViewData.Mapped Custodian Account'].iloc[0] == smb['ViewData.Mapped Custodian Account'].iloc[0]) and (ob['ViewData.Currency'].iloc[0] == smb['ViewData.Currency'].iloc[0]) and (ob['ViewData.Source Combination Code'].iloc[0] == smb['ViewData.Source Combination Code'].iloc[0])):

                ob_breakid.append(ob['ViewData.BreakID'].values)
                smb_breakid.append(smb['ViewData.BreakID'].values)
            
    if len(ob_breakid)>0:
        final_smb_ob_table = pd.DataFrame(ob_breakid)
        final_smb_ob_table.columns = ['BreakID_OB']
        final_smb_ob_table['BreakID_SMB'] = smb_breakid
        final_smb_ob_table['BreakID_SMB'] = final_smb_ob_table['BreakID_SMB'].apply(lambda x: str(x).replace("[",''))
        final_smb_ob_table['BreakID_SMB'] = final_smb_ob_table['BreakID_SMB'].apply(lambda x: str(x).replace("]",''))
        final_smb_ob_table['BreakID_SMB'] = final_smb_ob_table['BreakID_SMB'].astype(int)
    else:
        final_smb_ob_table = pd.DataFrame()
else:
    final_smb_ob_table = pd.DataFrame()



In [134]:
final_smb_ob_table

,BreakID_OB,BreakID_SMB
0,1653645115,1648967636
1,1653981730,1648950872
2,1654050883,1654050853


In [135]:
ob_breakid = []
umb_breakid = []
for amount in umb_amount:
    ob = umb_ob_table[(umb_ob_table['ViewData.Net Amount Difference Absolute']==amount) & (umb_ob_table['ViewData.Status']=='OB')]
    umb = umb_ob_table[(umb_ob_table['ViewData.Net Amount Difference Absolute']==amount) & (umb_ob_table['ViewData.Status']=='UMB')]
#         if((ob.shape[0]==1) and (smb.shape[0]==1) and (ob['ViewData.Mapped Custodian Account'] == smb['ViewData.Mapped Custodian Account']) and (ob['ViewData.Currency'] == smb['ViewData.Currency']) and (ob['ViewData.Source Combination Code'] == smb['ViewData.Source Combination Code'])):

    if ob.shape[0]==1 and umb.shape[0]==1 :
#Change added on 17-12-2020 by Rohit to include filter on ob and smb. Below if statement is commented out and new if statement is included
        if((ob['ViewData.Mapped Custodian Account'].iloc[0] == umb['ViewData.Mapped Custodian Account'].iloc[0]) and (ob['ViewData.Currency'].iloc[0] == umb['ViewData.Currency'].iloc[0]) and (ob['ViewData.Source Combination Code'].iloc[0] == umb['ViewData.Source Combination Code'].iloc[0])):

            ob_breakid.append(ob['ViewData.BreakID'].values)
            umb_breakid.append(umb['ViewData.BreakID'].values)
            
if len(ob_breakid)>0:
    final_umb_ob_table = pd.DataFrame(ob_breakid)
    final_umb_ob_table.columns = ['BreakID_OB']
    final_umb_ob_table['BreakID_UMB'] = umb_breakid
    final_umb_ob_table['BreakID_UMB'] = final_umb_ob_table['BreakID_UMB'].apply(lambda x: str(x).replace("[",''))
    final_umb_ob_table['BreakID_UMB'] = final_umb_ob_table['BreakID_UMB'].apply(lambda x: str(x).replace("]",''))
    final_umb_ob_table['BreakID_UMB'] = final_umb_ob_table['BreakID_UMB'].astype(int)
else:
    final_umb_ob_table = pd.DataFrame()


## Experiment

In [248]:
umb2 = meo[meo['ViewData.Status'].isin(['OB','UMB'])]
umb2 = umb2.reset_index().drop('index',1)

umb2['ViewData.Net Amount Difference Absolute'] = np.round(umb2['ViewData.Net Amount Difference Absolute'],2)

In [249]:
umb2['filter_key'] = umb2['ViewData.Mapped Custodian Account'] + umb2['ViewData.Currency']

In [250]:
umb2['filter_key'].nunique()

80

In [251]:
def subSum(numbers,total):
    for length in range(1, 3):
        if len(numbers) < length or length < 1:
            return []
        for index,number in enumerate(numbers):
            if length == 1 and np.isclose(number, total, atol=0.05).any():
                return [number]
            subset = subSum(numbers[index+1:],total-number)
            if subset: 
                return [number] + subset
        return []

In [284]:
break_id_aggregation = []
umb2_ids=[]
ob2_ids=[]


for key in umb2['filter_key'].unique():
    partial_data = umb2[umb2['filter_key']==key]

    if partial_data.shape[0]<=15 and any(x=='UMB' for x in partial_data['ViewData.Status'].values):
        values = partial_data['ViewData.Net Amount Difference']
        net_sum= 0
        
        if subSum(values,net_sum) == []: 
            #print("There are no valid subsets.")
            amount_array = ['NULL']
        else:
            amount_array = subSum(values,net_sum)

            break_id = partial_data[(partial_data['ViewData.Net Amount Difference'].isin(amount_array))]['ViewData.BreakID'].values
            #id0_unique = key
            
            for i in break_id:
                if partial_data[(partial_data['ViewData.BreakID']==i) & (partial_data['ViewData.Status']=='UMB')].shape[0]>0:
                    umb2_ids.append(i)
                else:
                    ob2_ids.append(i)
            break_id_aggregation.append(break_id)

In [288]:
pd.DataFrame(break_id_aggregation)

,0,1,2
0,1654004372,1654004362,1.649154e+09
1,1653645112,1653645132,NaN


In [286]:
umb2_ids

[1649153998]

In [287]:
ob2_ids

[1654004372, 1654004362, 1653645112, 1653645132]

In [260]:

umb2[umb2['ViewData.BreakID']==1649153998]['ViewData.Side0_UniqueIds']

550    77_3791122313_Advent Geneva,39_3791123345_Adve...
Name: ViewData.Side0_UniqueIds, dtype: object

## Experiment Ends

In [136]:
final_umb_ob_table

""


In [137]:
meo[meo['ViewData.BreakID']==1669576178][['ViewData.Net Amount Difference Absolute','ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds']]

,ViewData.Net Amount Difference Absolute,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds


In [138]:
meo[meo['ViewData.BreakID']==1607245427][['ViewData.Net Amount Difference Absolute','ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds']]

,ViewData.Net Amount Difference Absolute,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds
282,333.33,4_3791102560_Advent Geneva,5_3791104392_JP Morgan


In [139]:
final_umb_ob_table

""


In [140]:
#Remove BreakIDs caught in final_smb_ob_table if final_smb_ob_table is not null
if(final_smb_ob_table.shape[0] != 0):
    final_smb_ob_table['BreakID_SMB'] = final_smb_ob_table['BreakID_SMB'].astype(np.int64)
    final_smb_ob_table['BreakID_OB'] = final_smb_ob_table['BreakID_OB'].astype(np.int64)
    
    final_smb_ob_table_BreakID_list =  list(final_smb_ob_table['BreakID_OB']) + list(final_smb_ob_table['BreakID_SMB'])
    meo = meo[~meo['ViewData.BreakID'].isin(final_smb_ob_table_BreakID_list)]
else:
    final_smb_ob_table_BreakID_list = []
#End change code made on 12-12-2020


In [141]:
#Change made on 12-12-2020 to incorporate final_smb_ob_table. The BreakIDs in this table will be given the Predicted_Status of UMR and Predicted_action of UMR_One-Many_to_Many-One
#Begin code change made on 12-12-2020 to incorporate final_smb_ob_table
#final_smb_ob_table
if(final_smb_ob_table.shape[0] != 0):

    final_smb_ob_table_copy = pd.merge(final_smb_ob_table,meo_df[['ViewData.BreakID','ViewData.Task ID','ViewData.Task Business Date','ViewData.Source Combination Code']].drop_duplicates(), left_on = 'BreakID_OB',right_on = 'ViewData.BreakID', how='left')
    final_smb_ob_table_copy.drop('ViewData.BreakID', axis = 1, inplace = True)
    
    final_smb_ob_table_copy['Predicted_Status'] = 'UMR'
    final_smb_ob_table_copy['Predicted_action'] = 'UMR_One-Many_to_Many-One'
    final_smb_ob_table_copy['ML_flag'] = 'ML'
    final_smb_ob_table_copy['SetupID'] = setup_code 
    final_smb_ob_table_copy['ViewData.Task Business Date'] = pd.to_datetime(final_smb_ob_table_copy['ViewData.Task Business Date'])
    final_smb_ob_table_copy['ViewData.Task Business Date'] = final_smb_ob_table_copy['ViewData.Task Business Date'].map(lambda x: dt.datetime.strftime(x, '%Y-%m-%dT%H:%M:%SZ'))
    final_smb_ob_table_copy['ViewData.Task Business Date'] = pd.to_datetime(final_smb_ob_table_copy['ViewData.Task Business Date'])
    final_smb_ob_table_copy = make_Side0_Side1_columns_for_final_smb_ob_table(final_smb_ob_table_copy,meo_df)
    final_smb_ob_table_copy['probability_No_pair'] = ''
    final_smb_ob_table_copy['probability_UMB'] = ''
    final_smb_ob_table_copy['probability_UMR'] = ''
    final_smb_ob_table_copy['probability_UMT'] = ''
    final_smb_ob_table_copy['PredictedComment'] = ''
    final_smb_ob_table_copy['PredictedCategory'] = ''
    columns_rename_for_smb_ob_table_dict = {'BreakID_OB' : 'BreakID',
                                       'BreakID_SMB' : 'Final_predicted_break',
                                       'ViewData.Task ID' : 'TaskID',
                                       'ViewData.Task Business Date' : 'BusinessDate',
                                       'ViewData.Source Combination Code' : 'SourceCombinationCode'
                                       }
    final_smb_ob_table_copy.rename(columns = columns_rename_for_smb_ob_table_dict, inplace = True)
    filepaths_final_smb_ob_table_copy = '\\\\vitblrdevcons01\\Raman  Strategy ML 2.0\\All_Data\\' + client + '\\final_smb_ob_table_copy_setup_' + setup_code + '_date_' + str(date_i) + '.csv'

    final_smb_ob_table_copy.to_csv(filepaths_final_smb_ob_table_copy)


else:
    final_smb_ob_table_copy = pd.DataFrame()
#End code change made on 12-12-2020 to incorporate final_smb_ob_table


NameError: name 'meo_df' is not defined

In [142]:
final_smb_ob_table_copy

""


In [171]:

Side_0_1_UniqueIds_closed_all_dates_list = []


In [143]:

i = 0
#for i in range(0,len(date_numbers_list)):

#    Side_0_1_UniqueIds_closed_all_dates_list.append(
#            closed_daily_run(fun_setup_code=setup_code,\
#                             fun_date = i,\
#                             fun_meo_df_daily_run = meo)
#                             fun_main_filepath_meo= filepaths_MEO[i],\
#                             fun_main_filepath_aua = filepaths_AUA[i])
 #           )

#new_closed_keys = [i.replace('nan','') for i in Side_0_1_UniqueIds_closed_all_dates_list[0]]
#new_closed_keys = [i.replace('None','') for i in new_closed_keys]


df1 = meo[~meo['ViewData.Status'].isin(['SMT','HST', 'OC', 'CT', 'Archive','SMR'])]
#df = df[df['MatchStatus'] != 21]
df1 = df1[~df1['ViewData.Status'].isnull()]
df1 = df1.reset_index()
df1 = df1.drop('index',1)

## Output for Closed breaks

#closed_df_side1 = df1[df1['ViewData.Side1_UniqueIds'].isin(new_closed_keys)]
#closed_df_side0 = df1[df1['ViewData.Side0_UniqueIds'].isin(new_closed_keys)]
#closed_df = closed_df_side1.append(closed_df_side0)


In [144]:
#df2 = df1[~((df1['ViewData.Side1_UniqueIds'].isin(new_closed_keys)) | (df1['ViewData.Side0_UniqueIds'].isin(new_closed_keys)))]

df2 = df1.copy()
df = df2.copy()
df = df.reset_index()
df = df.drop('index',1)
df['Date'] = pd.to_datetime(df['ViewData.Task Business Date'])
df = df[~df['Date'].isnull()]
df = df.reset_index()
df = df.drop('index',1)

pd.to_datetime(df['Date'])

df['Date'] = pd.to_datetime(df['Date']).dt.date

df['Date'] = df['Date'].astype(str)

df = df[df['ViewData.Status'].isin(['OB','SDB','UOB','UDB','CMF','CNF','SMB','SPM'])]
df = df.reset_index()
df = df.drop('index',1)
df['ViewData.Side0_UniqueIds'] = df['ViewData.Side0_UniqueIds'].astype(str)
df['ViewData.Side1_UniqueIds'] = df['ViewData.Side1_UniqueIds'].astype(str)
df['flag_side0'] = df.apply(lambda x: len(x['ViewData.Side0_UniqueIds'].split(',')), axis=1)
df['flag_side1'] = df.apply(lambda x: len(x['ViewData.Side1_UniqueIds'].split(',')), axis=1)
df = df.rename(columns= {'ViewData.Cust Net Amount':'ViewData.B-P Net Amount'})

print('The Date value count is:')
print(df['Date'].value_counts())

date_i = df['Date'].mode()[0]

print('Choosing the date : ' + date_i)

sample = df[df['Date'] == date_i]
sample = sample.reset_index()
sample = sample.drop('index',1)


The Date value count is:
2020-12-08    690
2019-04-22      2
2020-03-04      2
2020-09-25      1
2020-04-15      1
2020-08-31      1
2020-07-07      1
2020-12-02      1
2020-12-04      1
2020-02-10      1
2020-11-19      1
2020-11-27      1
2020-09-02      1
2020-09-16      1
2020-11-30      1
Name: Date, dtype: int64
Choosing the date : 2020-12-08


In [145]:
smb = sample[sample['ViewData.Status']=='SMB'].reset_index()
smb = smb.drop('index',1)
smb_pb = smb.copy()
smb_acc = smb.copy()
smb_pb['ViewData.Accounting Net Amount'] = np.nan
smb_pb['ViewData.Side0_UniqueIds'] = np.nan
smb_pb['ViewData.Status'] ='SMB-OB'

smb_acc['ViewData.B-P Net Amount'] = np.nan
smb_acc['ViewData.Side1_UniqueIds'] = np.nan
smb_acc['ViewData.Status'] ='SMB-OB'

sample = sample[sample['ViewData.Status']!='SMB']
sample = sample.reset_index()
sample = sample.drop('index',1)

sample = pd.concat([sample,smb_pb,smb_acc],axis=0)
sample = sample.reset_index()
sample = sample.drop('index',1)

sample['ViewData.Side0_UniqueIds'] = sample['ViewData.Side0_UniqueIds'].astype(str)
sample['ViewData.Side1_UniqueIds'] = sample['ViewData.Side1_UniqueIds'].astype(str)


In [146]:
sample[sample['ViewData.Side1_UniqueIds']=='nan'].shape

(363, 375)

In [147]:
sample[sample['ViewData.Side0_UniqueIds']=='None'].shape

(0, 375)

In [148]:
sample[sample['ViewData.Side0_UniqueIds']=='NaN'].shape

(0, 375)

In [149]:
sample[sample['ViewData.Side0_UniqueIds']==''].shape

(0, 375)

In [150]:
sample.loc[sample['ViewData.Side0_UniqueIds']=='nan','flag_side0'] = 0
sample.loc[sample['ViewData.Side1_UniqueIds']=='nan','flag_side1'] = 0

sample.loc[sample['ViewData.Side0_UniqueIds']=='None','flag_side0'] = 0
sample.loc[sample['ViewData.Side1_UniqueIds']=='None','flag_side1'] = 0


sample.loc[sample['ViewData.Side0_UniqueIds']=='','flag_side0'] = 0
sample.loc[sample['ViewData.Side1_UniqueIds']=='','flag_side1'] = 0

sample.loc[sample['ViewData.Side1_UniqueIds']=='nan','Trans_side'] = 'B_side'
sample.loc[sample['ViewData.Side0_UniqueIds']=='nan','Trans_side'] = 'A_side'

sample.loc[sample['ViewData.Side1_UniqueIds']=='None','Trans_side'] = 'B_side'
sample.loc[sample['ViewData.Side0_UniqueIds']=='None','Trans_side'] = 'A_side'

sample.loc[sample['ViewData.Side1_UniqueIds']=='','Trans_side'] = 'B_side'
sample.loc[sample['ViewData.Side0_UniqueIds']=='','Trans_side'] = 'A_side'


In [151]:
sample.loc[sample['Trans_side']=='A_side','ViewData.B-P Currency'] = sample.loc[sample['Trans_side']=='A_side','ViewData.Currency']
sample.loc[sample['Trans_side']=='B_side','ViewData.Accounting Currency'] = sample.loc[sample['Trans_side']=='B_side','ViewData.Currency'] 

sample['ViewData.B-P Currency'] = sample['ViewData.B-P Currency'].astype(str)
sample['ViewData.Accounting Currency'] = sample['ViewData.Accounting Currency'].astype(str)
sample['ViewData.Mapped Custodian Account'] = sample['ViewData.Mapped Custodian Account'].astype(str)
sample['filter_key'] = sample.apply(lambda x: x['ViewData.Mapped Custodian Account'] + x['ViewData.B-P Currency'] if x['Trans_side']=='A_side' else x['ViewData.Mapped Custodian Account'] + x['ViewData.Accounting Currency'], axis=1)

sample1 = sample[(sample['flag_side0']<=1) & (sample['flag_side1']<=1) & (sample['ViewData.Status'].isin(['OB','SPM','SDB','UDB','UOB','SMB-OB','CNF','CMF']))]

sample1 = sample1.reset_index()
sample1 = sample1.drop('index', 1)

sample1['ViewData.BreakID'] = sample1['ViewData.BreakID'].astype(int)

sample1 = sample1[sample1['ViewData.BreakID']!=-1]
sample1 = sample1.reset_index()
sample1 = sample1.drop('index',1)

sample1 = sample1.sort_values(['ViewData.BreakID','Date'], ascending =[True, False])
sample1 = sample1.reset_index()
sample1 = sample1.drop('index',1)

aa = sample1[sample1['Trans_side']=='A_side']
bb = sample1[sample1['Trans_side']=='B_side']

aa['filter_key'] = aa['ViewData.Source Combination'].astype(str) + aa['ViewData.Mapped Custodian Account'].astype(str) + aa['ViewData.B-P Currency'].astype(str)

bb['filter_key'] = bb['ViewData.Source Combination'].astype(str) + bb['ViewData.Mapped Custodian Account'].astype(str) + bb['ViewData.Accounting Currency'].astype(str)

aa = aa.reset_index()
aa = aa.drop('index', 1)
bb = bb.reset_index()
bb = bb.drop('index', 1)

bb = bb[~bb['ViewData.Accounting Net Amount'].isnull()]
bb = bb.reset_index()
bb = bb.drop('index',1)
###################### loop m*n ###############################



pool =[]
key_index =[]
training_df =[]

no_pair_ids = []
#max_rows = 5

for d in (aa['Date'].unique()):
    aa1 = aa.loc[aa['Date']==d,:][common_cols]
    bb1 = bb.loc[bb['Date']==d,:][common_cols]
    
    aa1 = aa1.reset_index()
    aa1 = aa1.drop('index',1)
    bb1 = bb1.reset_index()
    bb1 = bb1.drop('index', 1)
    
    bb1 = bb1.sort_values(by='filter_key',ascending =True)
    
    for key in (list(np.unique(np.array(list(aa1['filter_key'].values) + list(bb1['filter_key'].values))))):
        
        df1 = aa1[aa1['filter_key']==key]
        df2 = bb1[bb1['filter_key']==key]

        if df1.empty == False and df2.empty == False:
            #aa_df = pd.concat([aa1[aa1.index==i]]*repeat_num, ignore_index=True)
            #bb_df = bb1.loc[pool[len(pool)-1],:][common_cols].reset_index()
            #bb_df = bb_df.drop('index', 1)

            df1 = df1.rename(columns={'ViewData.BreakID':'ViewData.BreakID_A_side'})
            df2 = df2.rename(columns={'ViewData.BreakID':'ViewData.BreakID_B_side'})

            #dff  = pd.concat([aa[aa.index==i],bb.loc[pool[i],:][accounting_vars]],axis=1)

            df1 = df1.reset_index()
            df2 = df2.reset_index()
            df1 = df1.drop('index', 1)
            df2 = df2.drop('index', 1)

            df1.columns = ['SideA.' + x  for x in df1.columns] 
            df2.columns = ['SideB.' + x  for x in df2.columns]

            df1 = df1.rename(columns={'SideA.filter_key':'filter_key'})
            df2 = df2.rename(columns={'SideB.filter_key':'filter_key'})

            #dff = pd.concat([aa_df,bb_df],axis=1)
            dff = merge(df1, df2, on='filter_key')
            training_df.append(dff)
                #key_index.append(i)
            #else:
            #no_pair_ids.append([aa1[(aa1['filter_key']=='key') & (aa1['ViewData.Status'].isin(['OB','SDB']))]['ViewData.Side1_UniqueIds'].values[0]])
               # no_pair_ids.append(aa1[(aa1['filter_key']== key) & (aa1['ViewData.Status'].isin(['OB','SDB']))]['ViewData.Side1_UniqueIds'].values[0])
    
        else:
#Change made on 26-11-2020 to include CMF and CNF as well, as long as they are single sided. For now, we are assuming they are single sided and no other precaution has been made to explicitely include single sided CNF and CMF
#Change made as per above and commenting below on 26-11-2020
#            no_pair_ids.append([aa1[(aa1['filter_key']==key) & (aa1['ViewData.Status'].isin(['OB','SDB']))]['ViewData.Side1_UniqueIds'].values])
#            no_pair_ids.append([bb1[(bb1['filter_key']==key) & (bb1['ViewData.Status'].isin(['OB','SDB']))]['ViewData.Side0_UniqueIds'].values])
#Change made on 26-11-2020 to include CNF and CMF
            no_pair_ids.append([aa1[(aa1['filter_key']==key) & (aa1['ViewData.Status'].isin(['OB','SDB','CNF','CMF']))]['ViewData.Side1_UniqueIds'].values])
            no_pair_ids.append([bb1[(bb1['filter_key']==key) & (bb1['ViewData.Status'].isin(['OB','SDB','CNF','CMF']))]['ViewData.Side0_UniqueIds'].values])
            


if len(no_pair_ids) != 0:
    no_pair_ids = np.unique(np.concatenate(no_pair_ids,axis=1)[0])
    no_pair_ids_df = pd.DataFrame(no_pair_ids, columns = ['Side0_1_UniqueIds'])
#    no_pair_ids_df = pd.merge(no_pair_ids_df, meo_df[['ViewData.Side1_UniqueIds','ViewData.BreakID','ViewData.Task ID','ViewData.Task Business Date']].drop_duplicates(), left_on = 'Side0_1_UniqueIds',right_on = 'ViewData.Side1_UniqueIds', how='left')
#    no_pair_ids_df = pd.merge(no_pair_ids_df, meo_df[['ViewData.Side0_UniqueIds','ViewData.BreakID','ViewData.Task ID','ViewData.Task Business Date']].drop_duplicates(), left_on = 'Side0_1_UniqueIds',right_on = 'ViewData.Side0_UniqueIds', how='left')
#    #no_pair_ids_df = no_pair_ids_df.rename(columns={'0':'filter_key'})
#    no_pair_ids_df['Predicted_Status'] = 'OB'
#    no_pair_ids_df['Predicted_action'] = 'No-Pair'
#    no_pair_ids_df['probability_No_pair'] = 0.9933
#    no_pair_ids_df['probability_UMB'] = 0.0033
#    no_pair_ids_df['probability_UMR'] = 0.0033    
#    no_pair_ids_df['ML_flag'] = 'ML'
#    no_pair_ids_df['TaskID'] = setup_code 
 #   no_pair_ids_df.to_csv(filepaths_no_pair_id_data)
#else:
#     with open(filepaths_no_pair_id_no_data_warning, 'w') as f:
#         f.write('No no pair ids found for this setup and date combination')


test_file = pd.concat(training_df)

test_file = test_file.reset_index()
test_file = test_file.drop('index',1)


C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [152]:
test_file['SideB.ViewData.BreakID_B_side'] = test_file['SideB.ViewData.BreakID_B_side'].astype('int64')
test_file['SideA.ViewData.BreakID_A_side'] = test_file['SideA.ViewData.BreakID_A_side'].astype('int64')

test_file['SideB.ViewData.CUSIP'] = test_file['SideB.ViewData.CUSIP'].str.split(".",expand=True)[0]
test_file['SideA.ViewData.CUSIP'] = test_file['SideA.ViewData.CUSIP'].str.split(".",expand=True)[0]

test_file['SideA.ViewData.ISIN'] = test_file['SideA.ViewData.ISIN'].astype(str)
test_file['SideB.ViewData.ISIN'] = test_file['SideB.ViewData.ISIN'].astype(str)
test_file['SideA.ViewData.CUSIP'] = test_file['SideA.ViewData.CUSIP'].astype(str)
test_file['SideB.ViewData.CUSIP'] = test_file['SideB.ViewData.CUSIP'].astype(str)
test_file['SideA.ViewData.Currency'] = test_file['SideA.ViewData.Currency'].astype(str)
test_file['SideB.ViewData.Currency'] = test_file['SideB.ViewData.Currency'].astype(str)

test_file['SideA.ViewData.Trade Date'] = test_file['SideA.ViewData.Trade Date'].astype(str)
test_file['SideB.ViewData.Trade Date'] = test_file['SideB.ViewData.Trade Date'].astype(str)
test_file['SideA.ViewData.Settle Date'] = test_file['SideA.ViewData.Settle Date'].astype(str)
test_file['SideB.ViewData.Settle Date'] = test_file['SideB.ViewData.Settle Date'].astype(str)
test_file['SideA.ViewData.Fund'] = test_file['SideA.ViewData.Fund'].astype(str)
test_file['SideB.ViewData.Fund'] = test_file['SideB.ViewData.Fund'].astype(str)

values_ISIN_A_Side = test_file['SideA.ViewData.ISIN'].values
values_ISIN_B_Side = test_file['SideB.ViewData.ISIN'].values
#test_file['ISIN_match'] = vec_equals_fun(values_ISIN_A_Side,values_ISIN_B_Side)

values_CUSIP_A_Side = test_file['SideA.ViewData.CUSIP'].values
values_CUSIP_B_Side = test_file['SideB.ViewData.CUSIP'].values
#
# values_CUSIP_A_Side = test_file['SideA.ViewData.Currency'].values
# values_CUSIP_B_Side = test_file['SideB.ViewData.Currency'].values

values_Currency_match_A_Side = test_file['SideA.ViewData.Currency'].values
values_Currency_match_B_Side = test_file['SideA.ViewData.Currency'].values

values_Trade_Date_match_A_Side = test_file['SideA.ViewData.Trade Date'].values
values_Trade_Date_match_B_Side = test_file['SideB.ViewData.Trade Date'].values

values_Settle_Date_match_A_Side = test_file['SideA.ViewData.Settle Date'].values
values_Settle_Date_match_B_Side = test_file['SideB.ViewData.Settle Date'].values

values_Fund_match_A_Side = test_file['SideA.ViewData.Fund'].values
values_Fund_match_B_Side = test_file['SideB.ViewData.Fund'].values

test_file['ISIN_match'] = vec_equals_fun(values_ISIN_A_Side,values_ISIN_B_Side)
test_file['CUSIP_match'] = vec_equals_fun(values_CUSIP_A_Side,values_CUSIP_B_Side)
test_file['Currency_match'] = vec_equals_fun(values_Currency_match_A_Side,values_Currency_match_B_Side)
test_file['Trade_Date_match'] = vec_equals_fun(values_Trade_Date_match_A_Side,values_Trade_Date_match_B_Side)
test_file['Settle_Date_match'] = vec_equals_fun(values_Settle_Date_match_A_Side,values_Settle_Date_match_B_Side)
test_file['Fund_match'] = vec_equals_fun(values_Fund_match_A_Side,values_Fund_match_B_Side)

test_file['Amount_diff_1'] = test_file['SideA.ViewData.Accounting Net Amount'] - test_file['SideB.ViewData.B-P Net Amount']
test_file['Amount_diff_2'] = test_file['SideB.ViewData.Accounting Net Amount'] - test_file['SideA.ViewData.B-P Net Amount']


# ## Description code

#os.chdir('D:\\ViteosModel\\OakTree - Pratik Code')#
#print(os.getcwd())

## TODO - Import a csv file for description category mapping

com = pd.read_csv('desc cat with naveen oaktree.csv')
#'desc cat with naveen oaktree.csv

cat_list = list(set(com['Pairing']))



test_file['SideA.desc_cat'] = test_file['SideA.ViewData.Description'].apply(lambda x : descclean(x,cat_list))
test_file['SideB.desc_cat'] = test_file['SideB.ViewData.Description'].apply(lambda x : descclean(x,cat_list))

test_file['SideA.desc_cat'] = test_file['SideA.desc_cat'].apply(lambda x : currcln(x))
test_file['SideB.desc_cat'] = test_file['SideB.desc_cat'].apply(lambda x : currcln(x))

com = com.drop(['var','Catogery'], axis = 1)

com = com.drop_duplicates()

com['Pairing'] = com['Pairing'].apply(lambda x : x.lower())
com['replace'] = com['replace'].apply(lambda x : x.lower())


test_file['SideA.new_desc_cat'] = test_file['SideA.desc_cat'].apply(lambda x : catcln1(x,com))
test_file['SideB.new_desc_cat'] = test_file['SideB.desc_cat'].apply(lambda x : catcln1(x,com))

comp = ['inc','stk','corp ','llc','pvt','plc']
test_file['SideA.new_desc_cat'] = test_file['SideA.new_desc_cat'].apply(lambda x : 'Company' if x in comp else x)

test_file['SideB.new_desc_cat'] = test_file['SideB.new_desc_cat'].apply(lambda x : 'Company' if x in comp else x)

test_file['SideA.new_desc_cat'] = test_file['SideA.new_desc_cat'].apply(lambda x : desccat(x))
test_file['SideB.new_desc_cat'] = test_file['SideB.new_desc_cat'].apply(lambda x : desccat(x))
# ## Prime Broker
test_file['new_pb'] = test_file['SideA.ViewData.Mapped Custodian Account'].apply(lambda x : x.split('_')[0] if type(x)==str else x)
new_pb_mapping = {'GSIL':'GS','CITIGM':'CITI','JPMNA':'JPM'}
test_file['SideA.ViewData.Prime Broker'] = test_file['SideA.ViewData.Prime Broker'].fillna('kkk')
test_file['new_pb1'] = test_file.apply(lambda x : x['new_pb'] if x['SideA.ViewData.Prime Broker']=='kkk' else x['SideA.ViewData.Prime Broker'],axis = 1)
test_file['Trade_date_diff'] = (pd.to_datetime(test_file['SideA.ViewData.Trade Date']) - pd.to_datetime(test_file['SideB.ViewData.Trade Date'])).dt.days

test_file['Settle_date_diff'] = (pd.to_datetime(test_file['SideA.ViewData.Settle Date']) - pd.to_datetime(test_file['SideB.ViewData.Settle Date'])).dt.days

############ Fund match new ########

values_Fund_match_A_Side = test_file['SideA.ViewData.Fund'].values
values_Fund_match_B_Side = test_file['SideB.ViewData.Fund'].values

vec_fund_match = np.vectorize(fundmatch)

test_file['SideA.ViewData.Fund'] = vec_fund_match(values_Fund_match_A_Side)
test_file['SideB.ViewData.Fund'] = vec_fund_match(values_Fund_match_B_Side)

### New code for cleaning text variables 
trans_type_A_side = test_file['SideA.ViewData.Transaction Type']
trans_type_B_side = test_file['SideB.ViewData.Transaction Type']

asset_type_cat_A_side = test_file['SideA.ViewData.Asset Type Category']
asset_type_cat_B_side = test_file['SideB.ViewData.Asset Type Category']

invest_type_A_side = test_file['SideA.ViewData.Investment Type']
invest_type_B_side = test_file['SideB.ViewData.Investment Type']

prime_broker_A_side = test_file['SideA.ViewData.Prime Broker']
prime_broker_B_side = test_file['SideB.ViewData.Prime Broker']

# LOWER CASE
trans_type_A_side = [str(item).lower() for item in trans_type_A_side]
trans_type_B_side = [str(item).lower() for item in trans_type_B_side]

asset_type_cat_A_side = [str(item).lower() for item in asset_type_cat_A_side]
asset_type_cat_B_side = [str(item).lower() for item in asset_type_cat_B_side]

invest_type_A_side = [str(item).lower() for item in invest_type_A_side]
invest_type_B_side = [str(item).lower() for item in invest_type_B_side]

prime_broker_A_side = [str(item).lower() for item in prime_broker_A_side]
prime_broker_B_side = [str(item).lower() for item in prime_broker_B_side]

split_trans_A_side = [item.split() for item in trans_type_A_side]
split_trans_B_side = [item.split() for item in trans_type_B_side]

split_asset_A_side = [item.split() for item in asset_type_cat_A_side]
split_asset_B_side = [item.split() for item in asset_type_cat_B_side]

split_invest_A_side = [item.split() for item in invest_type_A_side]
split_invest_B_side = [item.split() for item in invest_type_B_side]

split_prime_A_side = [item.split() for item in prime_broker_A_side]
split_prime_b_side = [item.split() for item in prime_broker_B_side]

## Transacion type

remove_nums_A_side = [[item for item in sublist if not is_num(item)] for sublist in split_trans_A_side]
remove_nums_B_side = [[item for item in sublist if not is_num(item)] for sublist in split_trans_B_side]

remove_dates_A_side = [[item for item in sublist if not (is_date_format(item) or date_edge_cases(item))] for sublist in remove_nums_A_side]
remove_dates_B_side = [[item for item in sublist if not (is_date_format(item) or date_edge_cases(item))] for sublist in remove_nums_B_side]


# Specific to clients already used on, will have to be edited for other edge cases
remove_amts_A_side = [[item for item in sublist if item[0] != '$'] for sublist in remove_dates_A_side]
remove_amts_B_side = [[item for item in sublist if item[0] != '$'] for sublist in remove_dates_B_side]


clean_adr_A_side = [(['ADR'] if 'adr' in item else item) for item in remove_amts_A_side]
clean_adr_B_side = [(['ADR'] if 'adr' in item else item) for item in remove_amts_B_side]

clean_tax_A_side = [(item[:2] if '30%' in item else item) for item in clean_adr_A_side]
clean_tax_B_side = [(item[:2] if '30%' in item else item) for item in clean_adr_B_side]

remove_ons_A_side = [(item[:item.index('on')] if 'on' in item else item) for item in clean_tax_A_side]
remove_ons_B_side = [(item[:item.index('on')] if 'on' in item else item) for item in clean_tax_B_side]

clean_eqswap_A_side = [(item[1:] if 'eqswap' in item else item) for item in remove_ons_A_side]
clean_eqswap_B_side = [(item[1:] if 'eqswap' in item else item) for item in remove_ons_B_side]

remove_mh_A_side = [[item for item in sublist if 'mh' not in item] for sublist in clean_eqswap_A_side]
remove_mh_B_side = [[item for item in sublist if 'mh' not in item] for sublist in clean_eqswap_B_side]

remove_ats_A_side = [(item[:item.index('@')] if '@' in item else item) for item in remove_mh_A_side]
remove_ats_B_side = [(item[:item.index('@')] if '@' in item else item) for item in remove_mh_B_side]

cleaned_trans_types_A_side = [' '.join(item) for item in remove_ats_A_side]
cleaned_trans_types_B_side = [' '.join(item) for item in remove_ats_B_side]

# # INVESTMENT TYPE

remove_nums_i_A_side = [[item for item in sublist if not is_num(item)] for sublist in split_invest_A_side]
remove_nums_i_B_side = [[item for item in sublist if not is_num(item)] for sublist in split_invest_B_side]

remove_dates_i_A_side = [[item for item in sublist if not is_date_format(item)] for sublist in remove_nums_i_A_side]
remove_dates_i_B_side = [[item for item in sublist if not is_date_format(item)] for sublist in remove_nums_i_B_side]

cleaned_invest_A_side = [' '.join(item) for item in remove_dates_i_A_side]
cleaned_invest_B_side = [' '.join(item) for item in remove_dates_i_B_side]

remove_nums_a_A_side = [[item for item in sublist if not is_num(item)] for sublist in split_asset_A_side]
remove_nums_a_B_side = [[item for item in sublist if not is_num(item)] for sublist in split_asset_B_side]

remove_dates_a_A_side = [[item for item in sublist if not is_date_format(item)] for sublist in remove_nums_a_A_side]
remove_dates_a_B_side = [[item for item in sublist if not is_date_format(item)] for sublist in remove_nums_a_B_side]

cleaned_asset_A_side = [' '.join(item) for item in remove_dates_a_A_side]
cleaned_asset_B_side = [' '.join(item) for item in remove_dates_a_B_side]

test_file['SideA.ViewData.Transaction Type'] = cleaned_trans_types_A_side
test_file['SideB.ViewData.Transaction Type'] = cleaned_trans_types_B_side

test_file['SideA.ViewData.Investment Type'] = cleaned_invest_A_side
test_file['SideB.ViewData.Investment Type'] = cleaned_invest_B_side

test_file['SideA.ViewData.Asset Category Type'] = cleaned_asset_A_side
test_file['SideB.ViewData.Asset Category Type'] = cleaned_asset_B_side

values_transaction_type_match_A_Side = test_file['SideA.ViewData.Transaction Type'].values
values_transaction_type_match_B_Side = test_file['SideB.ViewData.Transaction Type'].values

vec_tt_match = np.vectorize(mhreplaced)

test_file['SideA.ViewData.Transaction Type'] = vec_tt_match(values_transaction_type_match_A_Side)
test_file['SideB.ViewData.Transaction Type'] = vec_tt_match(values_transaction_type_match_B_Side)

test_file.loc[test_file['SideA.ViewData.Transaction Type']=='int','SideA.ViewData.Transaction Type'] = 'interest'
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='wires','SideA.ViewData.Transaction Type'] = 'wire'
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='dividends','SideA.ViewData.Transaction Type'] = 'dividend'
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='miscellaneous','SideA.ViewData.Transaction Type'] = 'misc'
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='div','SideA.ViewData.Transaction Type'] = 'dividend'

test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: x.replace('eqty','equity'))
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: x.replace('options','option'))
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: x.replace('eqt','equity'))
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: x.replace('eqty','equity'))

test_file['ViewData.Combined Transaction Type'] = test_file['SideA.ViewData.Transaction Type'].astype(str) +  test_file['SideB.ViewData.Transaction Type'].astype(str)
test_file['ViewData.Combined Fund'] = test_file['SideA.ViewData.Fund'].astype(str) + test_file['SideB.ViewData.Fund'].astype(str)

test_file['Combined_Investment_Type'] = test_file['SideA.ViewData.Investment Type'].astype(str) + test_file['SideB.ViewData.Investment Type'].astype(str)

test_file['Combined_Asset_Type_Category'] = test_file['SideA.ViewData.Asset Category Type'].astype(str) + test_file['SideB.ViewData.Asset Category Type'].astype(str)

    
vec_nan_fun = np.vectorize(nan_fun)
values_ISIN_A_Side = test_file['SideA.ViewData.ISIN'].values
values_ISIN_B_Side = test_file['SideB.ViewData.ISIN'].values
test_file['SideA.ISIN_NA'] = vec_nan_fun(values_ISIN_A_Side)
test_file['SideB.ISIN_NA'] = vec_nan_fun(values_ISIN_A_Side)

vec_a_key_match_fun = np.vectorize(a_keymatch)
vec_b_key_match_fun = np.vectorize(b_keymatch)

values_ISIN_A_Side = test_file['SideA.ViewData.ISIN'].values
values_ISIN_B_Side = test_file['SideB.ViewData.ISIN'].values

values_CUSIP_A_Side = test_file['SideA.ViewData.CUSIP'].values
values_CUSIP_B_Side = test_file['SideB.ViewData.CUSIP'].values

test_file['SideB.ViewData.key_NAN']= vec_a_key_match_fun(values_CUSIP_B_Side,values_ISIN_B_Side)[0]
test_file['SideB.ViewData.Common_key'] = vec_a_key_match_fun(values_CUSIP_B_Side,values_ISIN_B_Side)[1]
test_file['SideA.ViewData.key_NAN'] = vec_b_key_match_fun(values_CUSIP_A_Side,values_ISIN_A_Side)[0]
test_file['SideA.ViewData.Common_key'] = vec_b_key_match_fun(values_CUSIP_A_Side,values_ISIN_A_Side)[1]

vec_nan_equal_fun = np.vectorize(nan_equals_fun)
values_key_NAN_B_Side = test_file['SideB.ViewData.key_NAN'].values
values_key_NAN_A_Side = test_file['SideA.ViewData.key_NAN'].values
test_file['All_key_nan'] = vec_nan_equal_fun(values_key_NAN_B_Side,values_key_NAN_A_Side )

test_file['SideB.ViewData.Common_key'] = test_file['SideB.ViewData.Common_key'].astype(str)
test_file['SideA.ViewData.Common_key'] = test_file['SideA.ViewData.Common_key'].astype(str)

vec_new_key_match_fun = np.vectorize(new_key_match_fun)
values_Common_key_B_Side = test_file['SideB.ViewData.Common_key'].values
values_Common_key_A_Side = test_file['SideA.ViewData.Common_key'].values
values_All_key_NAN = test_file['All_key_nan'].values

test_file['new_key_match']= vec_new_key_match_fun(values_Common_key_B_Side,values_Common_key_A_Side,values_All_key_NAN)

test_file['amount_percent'] = (test_file['SideA.ViewData.B-P Net Amount']/test_file['SideB.ViewData.Accounting Net Amount']*100)

test_file['SideB.ViewData.Investment Type'] = test_file['SideB.ViewData.Investment Type'].apply(lambda x: str(x).lower())
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: str(x).lower())

test_file['SideB.ViewData.Prime Broker'] = test_file['SideB.ViewData.Prime Broker'].apply(lambda x: str(x).lower())
test_file['SideA.ViewData.Prime Broker'] = test_file['SideA.ViewData.Prime Broker'].apply(lambda x: str(x).lower())

test_file['SideB.ViewData.Asset Type Category'] = test_file['SideB.ViewData.Asset Type Category'].apply(lambda x: str(x).lower())
test_file['SideA.ViewData.Asset Type Category'] = test_file['SideA.ViewData.Asset Type Category'].apply(lambda x: str(x).lower())

test_file['ViewData.Combined Transaction Type'] = test_file['ViewData.Combined Transaction Type'].apply(lambda x: x.replace('jnl','journal'))

test_file['SideA.ViewData.Transaction Type'] = test_file['SideA.ViewData.Transaction Type'].apply(lambda x: x.replace('cover short','covershort'))

trade_types_A = ['buy', 'sell', 'covershort','sellshort',
       'fx', 'fx settlement', 'sell short',
       'trade not to be reported_buy', 'covershort','ptbl','ptss', 'ptcs', 'ptcl']
trade_types_B = ['trade not to be reported_buy','buy', 'sellshort', 'sell', 'covershort',
       'spotfx', 'forwardfx',
       'trade not to be reported_sell',
       'trade not to be reported_sellshort',
       'trade not to be reported_covershort']

test_file['SideA.TType'] = test_file.apply(lambda x: "Trade" if x['SideA.ViewData.Transaction Type'] in trade_types_A else "Non-Trade", axis=1)
test_file['SideB.TType'] = test_file.apply(lambda x: "Trade" if x['SideB.ViewData.Transaction Type'] in trade_types_B else "Non-Trade", axis=1)

test_file['Combined_Desc'] = test_file['SideA.new_desc_cat'] + test_file['SideB.new_desc_cat']

test_file['Combined_TType'] = test_file['SideA.TType'].astype(str) + test_file['SideB.TType'].astype(str)

for feature in ['SideA.Date','SideB.Date','SideA.ViewData.Settle Date','SideB.ViewData.Settle Date']:
    #train_full_new12[feature] = le.fit_transform(train_full_new12[feature])
    test_file[feature] = pd.to_datetime(test_file[feature],errors = 'coerce').dt.weekday



In [153]:

# ## UMR Mapping
## TODO Import HIstorical UMR FILE for Transaction Type mapping
oaktree_umr = pd.read_csv('OakTree_UMR.csv')

test_file['tt_map_flag'] = test_file.apply(lambda x: 1 if x['ViewData.Combined Transaction Type'] in oaktree_umr['ViewData.Combined Transaction Type'].unique() else 0, axis=1)

test_file['abs_amount_flag'] = test_file.apply(lambda x: 1 if x['SideB.ViewData.Accounting Net Amount'] == x['SideA.ViewData.B-P Net Amount']*(-1) else 0, axis=1)

test_file = test_file[~test_file['SideB.ViewData.Settle Date'].isnull()]
test_file = test_file[~test_file['SideA.ViewData.Settle Date'].isnull()]

test_file = test_file.reset_index().drop('index',1)
test_file['SideA.ViewData.Settle Date'] = test_file['SideA.ViewData.Settle Date'].astype(int)
test_file['SideB.ViewData.Settle Date'] = test_file['SideB.ViewData.Settle Date'].astype(int)


# ## Test file served into the model

test_file2 = test_file.copy()

X_test = test_file2[model_cols]

X_test = X_test.reset_index()
X_test = X_test.drop('index',1)
X_test = X_test.fillna(0)

X_test = X_test.fillna(0)

X_test.shape

X_test = X_test.drop_duplicates()
X_test = X_test.reset_index()
X_test = X_test.drop('index',1)

X_test.shape

# ## Model Pickle file import
## TODO Import Pickle file for 1st Model


(3005, 28)

In [154]:
X_test

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,...,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side
0,123085.00,119500.00,1,1,-3585.00,0.0,0,0,0,paydownpaydown,...,3,3,5fd051d515545831e8a1c686,5fd051d515545831e8a1c686,25_3791135288_Advent Geneva,15_3791133038_BNP Paribas,SMB-OB,1648950484,SMB-OB,1648950484
1,123085.00,21000.00,0,0,-102085.00,-1.0,-1,0,0,paydowninterest,...,3,4,5fd051d515545831e8a1c686,5fd051d515545831e8a1c66c,3_3791135288_Advent Geneva,15_3791133038_BNP Paribas,OB,1648950497,SMB-OB,1648950484
2,123085.00,3585.00,1,1,-119500.00,0.0,0,0,0,paydowncorp,...,3,3,5fd051d515545831e8a1c686,5fd051d515545831e8a1c66d,3_3791139052_Advent Geneva,15_3791133038_BNP Paribas,SMB-OB,1653591430,SMB-OB,1648950484
3,1946.85,119500.00,1,1,117553.15,0.0,0,0,0,corppaydown,...,3,3,5fd051d515545831e8a1c66d,5fd051d515545831e8a1c686,25_3791135288_Advent Geneva,14_3791133038_BNP Paribas,SMB-OB,1648950484,SMB-OB,1653591430
4,1946.85,21000.00,0,0,19053.15,-1.0,-1,0,0,corpinterest,...,3,4,5fd051d515545831e8a1c66d,5fd051d515545831e8a1c66c,3_3791135288_Advent Geneva,14_3791133038_BNP Paribas,OB,1648950497,SMB-OB,1653591430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,139.01,434.08,0,0,295.07,33.0,53,1,1,sellinterest,...,1,4,5fd076e21554581778dd3df3,5fd076e21554581778dd3e00,1_3791001046_Advent Geneva,1_3791071149_US BANK,OB,1469956890,OB,1563203798
3001,139.01,581.58,1,0,442.57,0.0,20,1,1,sellpay,...,1,2,5fd076e21554581778dd3df3,5fd076e21554581778dd3df8,2_3791044155_Advent Geneva,1_3791071149_US BANK,OB,1527672096,OB,1563203798
3002,139.01,484.00,0,0,344.99,61.0,81,1,1,sellinterest,...,1,4,5fd076e21554581778dd3df3,5fd076e21554581778dd3dff,1_379962666_Advent Geneva,1_3791071149_US BANK,OB,1420803037,OB,1563203798
3003,139.01,476.72,0,0,337.71,-61.0,-41,1,1,sellinterest,...,1,0,5fd076e21554581778dd3df3,5fd076e21554581778dd3dfd,1_3791126966_Advent Geneva,1_3791071149_US BANK,OB,1636975364,OB,1563203798


In [155]:
filename = 'OakTree_final_model2.sav'

clf = pickle.load(open(filename, 'rb'))

# ## Predictions

# Actual class predictions
rf_predictions = clf.predict(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))
# Probabilities for each class
rf_probs = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 1]

probability_class_0 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side','SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 0]
probability_class_1 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 1]

probability_class_2 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side','SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 2]

X_test['Predicted_action'] = rf_predictions
X_test['probability_No_pair'] = probability_class_0
X_test['probability_UMB'] = probability_class_1
X_test['probability_UMR'] = probability_class_2
X_test['Predicted_action'].value_counts()

# ## Two Step Modeling

X_test2 = test_file[model_cols_2]
X_test2 = X_test2.reset_index()
X_test2 = X_test2.drop('index',1)
X_test2 = X_test2.fillna(0)

X_test2.shape
X_test2 = X_test2.drop_duplicates()
X_test2 = X_test2.reset_index()
X_test2 = X_test2.drop('index',1)

X_test2.shape

## TODO Import MOdel2 as per the two step modelling process

filename2 = 'OakTree_final_model2_step_two.sav'
clf2 = pickle.load(open(filename2, 'rb'))

# Actual class predictions
rf_predictions2 = clf2.predict(X_test2.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))

# Probabilities for each class
rf_probs2 = clf2.predict_proba(X_test2.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 1]

probability_class_0_two = clf2.predict_proba(X_test2.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side','SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 0]
probability_class_1_two = clf2.predict_proba(X_test2.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 1]

X_test2['Predicted_action_2'] = rf_predictions2
X_test2['probability_No_pair_2'] = probability_class_0_two
X_test2['probability_UMB_2'] = probability_class_1_two

X_test = pd.concat([X_test, X_test2[['Predicted_action_2','probability_No_pair_2','probability_UMB_2']]],axis=1)



In [156]:
#Changes made on 25-11-2020.
#filepaths_X_test = '\\\\vitblrdevcons01\\Raman  Strategy ML 2.0\\All_Data\\' + client + '\\X_Test_for_Pratik_setup_' + setup_code + '_date_' + str(date_i) + '_2.csv'
#X_test.to_csv(filepaths_X_test)

# ## New Aggregation
X_test['Tolerance_level'] = np.abs(X_test['probability_UMB_2'] - X_test['probability_No_pair_2'])
b_side_agg = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action_2'].unique().reset_index()
a_side_agg = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action_2'].unique().reset_index()


# ## UMR segregation
umr_ids_0 = umr_seg(X_test)


In [157]:
len(umr_ids_0)

22

In [158]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds'].isin(umr_ids_0) & (X_test['Predicted_action']=='UMR_One_to_One')].shape

(22, 36)

In [159]:
final_umr_table = X_test[X_test['SideB.ViewData.Side0_UniqueIds'].isin(umr_ids_0) & (X_test['Predicted_action']=='UMR_One_to_One')]


In [35]:
if(len(duplicate_ids_final_umr_table_Side0) != 0):
    final_umr_table_duplicates_Side0 = final_umr_table[final_umr_table['SideB.ViewData.Side0_UniqueIds'].isin(duplicate_ids_final_umr_table_Side0)]
    final_umr_table_duplicates_Side0 = final_umr_table_duplicates_Side0[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side','Predicted_action','probability_No_pair','probability_UMB','probability_UMR']]#,'probability_UMT']]
    final_umr_table = final_umr_table[~final_umr_table['SideB.ViewData.Side0_UniqueIds'].isin(duplicate_ids_final_umr_table_Side0)]
    final_umr_table_side0_ids = list(set(final_umr_table['SideB.ViewData.Side0_UniqueIds']))
    side0_umr_ids_to_remove_from_final_open_table = final_umr_table_side0_ids + list(duplicate_ids_final_umr_table_Side0)
    
else:
    final_umr_table_duplicates_Side0 = pd.DataFrame()
    final_umr_table_side0_ids = list(set(final_umr_table['SideB.ViewData.Side0_UniqueIds']))
    side0_umr_ids_to_remove_from_final_open_table = final_umr_table_side0_ids


NameError: name 'duplicate_ids_final_umr_table_Side0' is not defined

In [189]:
if(len(duplicate_ids_final_umr_table_Side1) != 0):
    final_umr_table_duplicates_Side1 = final_umr_table[final_umr_table['SideA.ViewData.Side1_UniqueIds'].isin(duplicate_ids_final_umr_table_Side1)]
    final_umr_table_duplicates_Side1 = final_umr_table_duplicates_Side1[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side','Predicted_action','probability_No_pair','probability_UMB','probability_UMR']]#,'probability_UMT']]
    final_umr_table = final_umr_table[~final_umr_table['SideA.ViewData.Side1_UniqueIds'].isin(duplicate_ids_final_umr_table_Side1)]
    final_umr_table_side1_ids = list(set(final_umr_table['SideA.ViewData.Side1_UniqueIds'])) 
    side1_umr_ids_to_remove_from_final_open_table = final_umr_table_side1_ids + list(duplicate_ids_final_umr_table_Side1)

else:
    final_umr_table_duplicates_Side1 = pd.DataFrame()
    final_umr_table_side1_ids = list(set(final_umr_table['SideA.ViewData.Side1_UniqueIds']))
    side1_umr_ids_to_remove_from_final_open_table = final_umr_table_side1_ids


In [190]:
len(side1_umr_ids_to_remove_from_final_open_table)

70

In [160]:
# ## 1st Prediction Table for One to One UMR
    

final_umr_table = final_umr_table[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side','Predicted_action','probability_No_pair','probability_UMB','probability_UMR']]

# ## No-Pair segregation

no_pair_ids_b_side, no_pair_ids_a_side = no_pair_seg(X_test)

X_test[(X_test['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side))]['Predicted_action_2'].value_counts()

X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action_2'].unique().reset_index()

X_test[X_test['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side)]['Predicted_action_2'].value_counts()

final_open_table = X_test[(X_test['SideB.ViewData.Side0_UniqueIds'].isin(no_pair_ids_b_side)) | (X_test['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side))]


In [37]:
final_open_table = final_open_table[~final_open_table['SideA.ViewData.Side1_UniqueIds'].isin(side1_umr_ids_to_remove_from_final_open_table)]
final_open_table = final_open_table[~final_open_table['SideB.ViewData.Side0_UniqueIds'].isin(side0_umr_ids_to_remove_from_final_open_table)]

final_open_table = final_open_table[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side','Predicted_action_2','probability_No_pair_2','probability_UMB_2','probability_UMR']]

final_open_table['probability_UMR'] = 0.00010
final_open_table = final_open_table.rename(columns = {'Predicted_action_2':'Predicted_action','probability_No_pair_2':'probability_No_pair','probability_UMB_2':'probability_UMB'})


b_side_open_table = final_open_table.groupby('SideB.ViewData.Side0_UniqueIds')[['probability_No_pair','probability_UMB','probability_UMR']].mean().reset_index()
a_side_open_table = final_open_table.groupby('SideA.ViewData.Side1_UniqueIds')[['probability_No_pair','probability_UMB','probability_UMR']].mean().reset_index()

a_side_open_table = a_side_open_table[a_side_open_table['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side)]
b_side_open_table = b_side_open_table[b_side_open_table['SideB.ViewData.Side0_UniqueIds'].isin(no_pair_ids_b_side)]

b_side_open_table = b_side_open_table.reset_index().drop('index',1)
a_side_open_table = a_side_open_table.reset_index().drop('index',1)

final_no_pair_table = pd.concat([a_side_open_table,b_side_open_table], axis=0)
final_no_pair_table = final_no_pair_table.reset_index().drop('index',1)


NameError: name 'side1_umr_ids_to_remove_from_final_open_table' is not defined

In [38]:
final_no_pair_table

NameError: name 'final_no_pair_table' is not defined

In [60]:
#
#final_no_pair_table = pd.merge(final_no_pair_table, final_open_table[['SideA.ViewData.Side1_UniqueIds','SideA.ViewData.BreakID_A_side']].drop_duplicates(), on = 'SideA.ViewData.Side1_UniqueIds', how='left')
#final_no_pair_table = pd.merge(final_no_pair_table, final_open_table[['SideB.ViewData.Side0_UniqueIds','SideB.ViewData.BreakID_B_side']].drop_duplicates(), on = 'SideB.ViewData.Side0_UniqueIds', how='left')
#

final_no_pair_table = normalize_final_no_pair_table_col_names(fun_final_no_pair_table = final_no_pair_table)
final_no_pair_table_copy = final_no_pair_table.copy()

final_no_pair_table_copy['ViewData.Side0_UniqueIds'] = final_no_pair_table_copy['ViewData.Side0_UniqueIds'].astype(str)
final_no_pair_table_copy['ViewData.Side1_UniqueIds'] = final_no_pair_table_copy['ViewData.Side1_UniqueIds'].astype(str)


In [61]:
final_no_pair_table_copy

,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,probability_No_pair,probability_UMB,probability_UMR
0,100_3791127028_The Bank of New York Mellon,nan,0.934582,0.065418,0.0001
1,100_3791149966_Royal Bank of Canada,nan,0.998973,0.001027,0.0001
2,101_3791149966_Royal Bank of Canada,nan,0.998908,0.001092,0.0001
3,102_3791149966_Royal Bank of Canada,nan,0.998973,0.001027,0.0001
4,103_3791149966_Royal Bank of Canada,nan,0.998908,0.001092,0.0001
...,...,...,...,...,...
756,nan,98_3791149966_Advent Geneva,0.998304,0.001696,0.0001
757,nan,99_3791149966_Advent Geneva,0.998304,0.001696,0.0001
758,nan,9_3791025073_Advent Geneva,0.997568,0.002432,0.0001
759,nan,9_3791149051_Advent Geneva,0.949525,0.050475,0.0001


In [202]:
 
final_no_pair_table_copy.loc[final_no_pair_table_copy['ViewData.Side0_UniqueIds']=='None','Side0_1_UniqueIds'] = final_no_pair_table_copy['ViewData.Side1_UniqueIds']
final_no_pair_table_copy.loc[final_no_pair_table_copy['ViewData.Side1_UniqueIds']=='None','Side0_1_UniqueIds'] = final_no_pair_table_copy['ViewData.Side0_UniqueIds']

final_no_pair_table_copy.loc[final_no_pair_table_copy['ViewData.Side0_UniqueIds']=='nan','Side0_1_UniqueIds'] = final_no_pair_table_copy['ViewData.Side1_UniqueIds']
final_no_pair_table_copy.loc[final_no_pair_table_copy['ViewData.Side1_UniqueIds']=='nan','Side0_1_UniqueIds'] = final_no_pair_table_copy['ViewData.Side0_UniqueIds']

del final_no_pair_table_copy['ViewData.Side0_UniqueIds']
del final_no_pair_table_copy['ViewData.Side1_UniqueIds']


In [161]:
#OTM,MTO,OTO code begin

# ## Remove Open Ids

umr_ids_a_side = final_umr_table['SideA.ViewData.Side1_UniqueIds'].unique()
umr_ids_b_side = final_umr_table['SideB.ViewData.Side0_UniqueIds'].unique()

### Remove Open IDs

X_test_left = X_test[~(X_test['SideB.ViewData.Side0_UniqueIds'].isin(no_pair_ids_b_side))]
X_test_left = X_test_left[~(X_test_left['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side))]

## Remove UMR IDs

X_test_left = X_test_left[~(X_test_left['SideA.ViewData.Side1_UniqueIds'].isin(umr_ids_a_side))]
X_test_left = X_test_left[~(X_test_left['SideB.ViewData.Side0_UniqueIds'].isin(umr_ids_b_side))]


X_test_left = X_test_left.reset_index().drop('index',1)

# ## One to One UMB segregation

X_test_left['Predicted_action_2'].value_counts()

### IDs left after removing UMR ids from 0 and 1 side

X_test_left = X_test_left[~(X_test_left['SideA.ViewData.Side1_UniqueIds'].isin(final_umr_table['SideA.ViewData.Side1_UniqueIds']))]

X_test_left = X_test_left[~(X_test_left['SideB.ViewData.Side0_UniqueIds'].isin(final_umr_table['SideB.ViewData.Side0_UniqueIds']))]

X_test_left.shape
X_test_left['Predicted_action_2'].value_counts()

X_test_left = X_test_left.drop(['SideB.ViewData._ID','SideA.ViewData._ID'],1).drop_duplicates()
X_test_left = X_test_left.reset_index().drop('index',1)

for key in X_test_left['SideB.ViewData.Side0_UniqueIds'].unique():
    umb_ids_1 = X_test_left[(X_test_left['SideB.ViewData.Side0_UniqueIds']==key) & (X_test_left['Predicted_action_2']=='UMB_One_to_One')]['SideA.ViewData.Side1_UniqueIds'].unique()

X_test_left['SideB.ViewData.Side0_UniqueIds'].value_counts()



62_3791114597_Advent Geneva     25
44_3791011889_Advent Geneva     25
60_3791001204_Advent Geneva     25
61_3791001204_Advent Geneva     25
64_3791114597_Advent Geneva     25
                                ..
107_3791006998_Advent Geneva     1
423_3791088743_Advent Geneva     1
14_3791006998_Advent Geneva      1
99_3791118350_Advent Geneva      1
11_3791118350_Advent Geneva      1
Name: SideB.ViewData.Side0_UniqueIds, Length: 168, dtype: int64

In [ ]:
# # Before changes on 17-12-2020
# # ## UMR One to Many and Many to One 

# # ### One to Many
# cliff_for_loop = 16

# threshold_0 = X_test['SideB.ViewData.Side0_UniqueIds'].value_counts().reset_index(name='count')
# threshold_0_umb = threshold_0[threshold_0['count']>cliff_for_loop]['index'].unique()
# threshold_0_without_umb = threshold_0[threshold_0['count']<=cliff_for_loop]['index'].unique()

# exceptions_0_umb = X_test[X_test['Predicted_action_2']=='UMB_One_to_One']['SideB.ViewData.Side0_UniqueIds'].value_counts().reset_index(name='count')
# exceptions_0_umb_ids = exceptions_0_umb[exceptions_0_umb['count']>cliff_for_loop]['index'].unique()

# many_ids_1 = []
# one_id_0 = []
# amount_array =[]
# for key in X_test[~((X_test['SideB.ViewData.Side0_UniqueIds'].isin(exceptions_0_umb_ids)) | (X_test['SideB.ViewData.Side0_UniqueIds'].isin(final_umr_table['SideB.ViewData.Side0_UniqueIds'])))]['SideB.ViewData.Side0_UniqueIds'].unique():
#     print(key)
    
#     if key in threshold_0_umb:

#         values =  X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key) & (X_test['Predicted_action_2']=='UMB_One_to_One')]['SideA.ViewData.B-P Net Amount'].values
#         net_sum = X_test[X_test['SideB.ViewData.Side0_UniqueIds']==key]['SideB.ViewData.Accounting Net Amount'].max()

#         #memo = dict()
#         #print(values)
#         #print(net_sum)

#         if subSum(values,net_sum) == []: 
#             #print("There are no valid subsets.")
#             amount_array = ['NULL']
#         else:
#             amount_array = subSum(values,net_sum)

#             id1_aggregation = X_test[(X_test['SideA.ViewData.B-P Net Amount'].isin(amount_array)) & (X_test['SideB.ViewData.Side0_UniqueIds']==key)]['SideA.ViewData.Side1_UniqueIds'].values
#             id0_unique = key       

#             if len(id1_aggregation)>1: 
#                 many_ids_1.append(id1_aggregation)
#                 one_id_0.append(id0_unique)
#             else:
#                 pass
            
#     else:
#         values =  X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key)]['SideA.ViewData.B-P Net Amount'].values
#         net_sum = X_test[X_test['SideB.ViewData.Side0_UniqueIds']==key]['SideB.ViewData.Accounting Net Amount'].max()

#         #memo = dict()
#         #print(values)
#         #print(net_sum)

#         if subSum(values,net_sum) == []: 
#             #print("There are no valid subsets.")
#             amount_array = ['NULL']
#         else:
#             amount_array = subSum(values,net_sum)

#             id1_aggregation = X_test[(X_test['SideA.ViewData.B-P Net Amount'].isin(amount_array)) & (X_test['SideB.ViewData.Side0_UniqueIds']==key)]['SideA.ViewData.Side1_UniqueIds'].values
#             id0_unique = key       

#             if len(id1_aggregation)>1: 
#                 many_ids_1.append(id1_aggregation)
#                 one_id_0.append(id0_unique)
#             else:
#                 pass

# umr_otm_table = pd.DataFrame(one_id_0)

# if(umr_otm_table.empty == False):
#     umr_otm_table.columns = ['SideB.ViewData.Side0_UniqueIds']
#     umr_otm_table['SideA.ViewData.Side1_UniqueIds'] =many_ids_1 
# else:
#     temp_umr_otm_table_message = 'No One to many found'
#     print(temp_umr_otm_table_message)


# # ## Removing duplicate IDs from side 1

# if(len(many_ids_1) == 0):
#     unique_many_ids_1 = ['None']
# else:
#     unique_many_ids_1 = np.unique(np.concatenate(many_ids_1))

# dup_ids_0 = []
# for i in unique_many_ids_1:
#     count =0
#     for j in many_ids_1:
#         if i in j:
#             count = count+1
#             if count==2:
#                 dup_ids_0.append(i)
#                 break             
            
# dup_array_0 = []
# for i in many_ids_1:
#     #print(i)
#     if any(x in dup_ids_0 for x in i):
#         dup_array_0.append(i)
        

# ### Converting array to list
# dup_array_0_list = []
# for i in dup_array_0:
#     dup_array_0_list.append(list(i))
    
# many_ids_1_list =[] 
# for j in many_ids_1:
#     many_ids_1_list.append(list(j))
    
    
# filtered_otm = [i for i in many_ids_1_list if not i in dup_array_0_list]

# one_id_0_final = []
# for i, j in zip(many_ids_1_list, one_id_0):
#     if i in filtered_otm:
#         one_id_0_final.append(j) 

# umr_otm_table = umr_otm_table[umr_otm_table['SideB.ViewData.Side0_UniqueIds'].isin(one_id_0_final)]

# filtered_otm_flat = [item for sublist in filtered_otm for item in sublist]


# # ## Including UMR double count into OTM
# umr_double_count = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].value_counts().reset_index(name='count')
# umr_double_count = umr_double_count[(umr_double_count['Predicted_action']=='UMR_One_to_One') & (umr_double_count['count']==2)]

# umr_double_count_left = umr_double_count[~umr_double_count['SideB.ViewData.Side0_UniqueIds'].isin(umr_otm_table['SideB.ViewData.Side0_UniqueIds'].unique())]

# pb_ids_otm_left = []
# acc_id_single = []

# for key in umr_double_count_left['SideB.ViewData.Side0_UniqueIds'].unique():
#     acc_amount = X_test[X_test['SideB.ViewData.Side0_UniqueIds']==key]['SideB.ViewData.Accounting Net Amount'].max()
#     pb_ids_otm = X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key) & ((X_test['SideB.ViewData.Accounting Net Amount']==X_test['SideA.ViewData.B-P Net Amount']) | (X_test['SideB.ViewData.Accounting Net Amount']== (-1)*X_test['SideA.ViewData.B-P Net Amount']))]['SideA.ViewData.Side1_UniqueIds'].values
#     pb_ids_otm_left.append(pb_ids_otm)
#     acc_id_single.append(key)

# umr_otm_table_double_count = pd.DataFrame(acc_id_single)
# if(umr_otm_table_double_count.shape[0] != 0):
#     umr_otm_table_double_count.columns = ['SideB.ViewData.Side0_UniqueIds']

#     umr_otm_table_double_count['SideA.ViewData.Side1_UniqueIds'] = pb_ids_otm_left

#     umr_otm_table_final = pd.concat([umr_otm_table, umr_otm_table_double_count], axis=0)
# else:
#     umr_otm_table_final = umr_otm_table.copy()
    
# if(umr_otm_table_final.empty == False):
#     umr_otm_table_final = umr_otm_table_final.reset_index().drop('index',1)


In [162]:
# After changes on 17-12-2020
cliff_for_loop = 16

threshold_0 = X_test['SideB.ViewData.Side0_UniqueIds'].value_counts().reset_index(name='count')
threshold_0_umb = threshold_0[threshold_0['count']>cliff_for_loop]['index'].unique()
threshold_0_without_umb = threshold_0[threshold_0['count']<=cliff_for_loop]['index'].unique()

exceptions_0_umb = X_test[X_test['Predicted_action_2']!='UMR_One_to_One']['SideB.ViewData.Side0_UniqueIds'].value_counts().reset_index(name='count')
exceptions_0_umb_ids = exceptions_0_umb[exceptions_0_umb['count']>cliff_for_loop]['index'].unique()

def subSum(numbers,total):
    for length in range(1, 3):
        if len(numbers) < length or length < 1:
            return []
        for index,number in enumerate(numbers):
            if length == 1 and np.isclose(number, total, atol=0.25).any():
                return [number]
            subset = subSum(numbers[index+1:],total-number)
            if subset: 
                return [number] + subset
        return []
               

#null_value ='No'
many_ids_1 = []
one_id_0 = []
amount_array =[]

loop_count = 0
for key in exceptions_0_umb['index'].unique():
#for key in ['553_1251128974_Advent Geneva','12_3791149966_Advent Geneva']:
    loop_count = loop_count + 1
    print(loop_count)
    if key in exceptions_0_umb_ids:
        sort_data = X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key) & (X_test['Predicted_action_2']!='UMR_One_to_One')].sort_values(by = ['probability_UMB_2'], ascending =[False])
        sort_data = sort_data.reset_index().drop('index',1)
#        Change made on 17-12-2020. As per Pratik, we will take the first 15 values, as oaktree is smaller data. For weiss, we take 8 as weiss is too big and it takes too much time with value of 15
#        sort_data = sort_data.loc[0:10,:]
        sort_data = sort_data.loc[0:10,:]
        sort_data = sort_data.drop_duplicates(subset=['SideA.ViewData.B-P Net Amount'])
        sort_data = sort_data.reset_index().drop('index',1)
        #print(sort_data)

        values =  sort_data['SideA.ViewData.B-P Net Amount'].values
        net_sum = sort_data['SideB.ViewData.Accounting Net Amount'].max()

        #memo = dict()
        #print(values)
        #print(net_sum)

        if subSum(values,net_sum) == []: 
            #print("There are no valid subsets.")
            amount_array = ['NULL']
        else:
            amount_array = subSum(values,net_sum)

            id1_aggregation = sort_data[(sort_data['SideA.ViewData.B-P Net Amount'].isin(amount_array)) & (sort_data['SideB.ViewData.Side0_UniqueIds']==key)]['SideA.ViewData.Side1_UniqueIds'].values
            id0_unique = key       

            if len(id1_aggregation)>1: 
                many_ids_1.append(id1_aggregation)
                one_id_0.append(id0_unique)
            else:
                pass
            
    else:
        
        sort_data2 = X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key) & (X_test['Predicted_action_2']!='UMR_One_to_One')].sort_values(by = ['probability_UMB_2'], ascending =[False])
        sort_data2 = sort_data2.reset_index().drop('index',1)

#        Change made on 08-12-2020. As per Pratik, we will take the first 8 values, not 10 in order to not overpredict otm and mto umrs
#        sort_data2 = sort_data2.loc[0:10,:]
        sort_data2 = sort_data2.loc[0:10,:]
        
        sort_data2 = sort_data2.drop_duplicates(subset=['SideA.ViewData.B-P Net Amount'])
        sort_data2 = sort_data2.reset_index().drop('index',1)
        

        values =  sort_data2['SideA.ViewData.B-P Net Amount'].values
        net_sum = sort_data2['SideB.ViewData.Accounting Net Amount'].max()
        #values =  X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key) & (X_test['Predicted_action_2']=='UMB_One_to_One')]['SideA.ViewData.B-P Net Amount'].values
        #net_sum = X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key)& (X_test['Predicted_action_2']=='UMB_One_to_One')]['SideB.ViewData.Accounting Net Amount'].max()

        #memo = dict()
        #print(values)
        #print(net_sum)

        if subSum(values,net_sum) == []: 
            #print("There are no valid subsets.")
            amount_array = ['NULL']
        else:
            amount_array = subSum(values,net_sum)

            id1_aggregation = sort_data2[(sort_data2['SideA.ViewData.B-P Net Amount'].isin(amount_array)) & (sort_data2['SideB.ViewData.Side0_UniqueIds']==key)]['SideA.ViewData.Side1_UniqueIds'].values
            id0_unique = key       

            if len(id1_aggregation)>1: 
                many_ids_1.append(id1_aggregation)
                one_id_0.append(id0_unique)
            else:
                pass

umr_otm_table = pd.DataFrame(one_id_0)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [163]:
#End change


if umr_otm_table.empty == False:
    umr_otm_table.columns = ['SideB.ViewData.Side0_UniqueIds']
    umr_otm_table['SideA.ViewData.Side1_UniqueIds'] =many_ids_1
else:
    print('No One to Many found')

In [164]:
umr_otm_table

,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds
0,7_3791139058_Advent Geneva,"[3_3791139058_Credit Suisse, 5_3791139058_Cred..."
1,8_3791139405_Advent Geneva,"[31_3791137060_Northern Trust, 30_3791137060_N..."
2,225_3791127144_Advent Geneva,"[112_3791139278_State Street, 111_3791139278_S..."
3,89_3791139278_Advent Geneva,"[112_3791139278_State Street, 111_3791139278_S..."
4,82_3791139278_Advent Geneva,"[112_3791139278_State Street, 111_3791139278_S..."
5,62_3791139105_Advent Geneva,"[97_3791133104_The Bank of New York Mellon, 2_..."
6,131_3791139278_Advent Geneva,"[50_3791139278_State Street, 48_3791139278_Sta..."
7,52_3791139278_Advent Geneva,"[68_3791137344_State Street, 67_3791137344_Sta..."
8,58_3791139278_Advent Geneva,"[71_3791137344_State Street, 70_3791137344_Sta..."


In [165]:
#X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='12_3791149966_Advent Geneva']

In [166]:


# ## Removing duplicate IDs from side 1

if len(many_ids_1)!=0:
    unique_many_ids_1 = np.unique(np.concatenate(many_ids_1))
else:
    unique_many_ids_1 = np.array(['None'])

dup_ids_0 = []
for i in unique_many_ids_1:
    count =0
    for j in many_ids_1:
        if i in j:
            count = count+1
            if count==2:
                dup_ids_0.append(i)
                break             
            
dup_array_0 = []
for i in many_ids_1:
    #print(i)
    if any(x in dup_ids_0 for x in i):
        dup_array_0.append(i)
        

### Converting array to list
dup_array_0_list = []
for i in dup_array_0:
    dup_array_0_list.append(list(i))
    
many_ids_1_list =[] 
for j in many_ids_1:
    many_ids_1_list.append(list(j))
    
    
filtered_otm = [i for i in many_ids_1_list if not i in dup_array_0_list]

one_id_0_final = []
for i, j in zip(many_ids_1_list, one_id_0):
    if i in filtered_otm:
        one_id_0_final.append(j) 

#meo[meo['ViewData.Side0_UniqueIds'] =='162_153156748_Advent Geneva']

if len(one_id_0_final)!=0:
    #unique_many_ids_1 = np.unique(np.concatenate(many_ids_1))
    one_id_0_final = one_id_0_final
else:
    one_id_0_final = np.array(['None'])
    
if umr_otm_table.empty == False:    
    umr_otm_table = umr_otm_table[umr_otm_table['SideB.ViewData.Side0_UniqueIds'].isin(one_id_0_final)]

filtered_otm_flat = [item for sublist in filtered_otm for item in sublist]

# ## Including UMR double count into OTM

umr_double_count = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].value_counts().reset_index(name='count')
umr_double_count = umr_double_count[(umr_double_count['Predicted_action']=='UMR_One_to_One') & (umr_double_count['count']==2)]


if umr_otm_table.empty == False:
    sideb_unique = umr_otm_table['SideB.ViewData.Side0_UniqueIds'].unique()
else:
    sideb_unique =['None']
if umr_double_count.empty == False:

    umr_double_count_left = umr_double_count[~umr_double_count['SideB.ViewData.Side0_UniqueIds'].isin(sideb_unique)]

pb_ids_otm_left = []
acc_id_single = []

for key in umr_double_count_left['SideB.ViewData.Side0_UniqueIds'].unique():
    acc_amount = X_test[X_test['SideB.ViewData.Side0_UniqueIds']==key]['SideB.ViewData.Accounting Net Amount'].max()
    pb_ids_otm = X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key) & ((X_test['SideB.ViewData.Accounting Net Amount']==X_test['SideA.ViewData.B-P Net Amount']) | (X_test['SideB.ViewData.Accounting Net Amount']== (-1)*X_test['SideA.ViewData.B-P Net Amount']))]['SideA.ViewData.Side1_UniqueIds'].values
    pb_ids_otm_left.append(pb_ids_otm)
    acc_id_single.append(key)
    
umr_otm_table_double_count = pd.DataFrame(acc_id_single)
umr_otm_table_double_count.columns = ['SideB.ViewData.Side0_UniqueIds']

umr_otm_table_double_count['SideA.ViewData.Side1_UniqueIds'] = pb_ids_otm_left

umr_otm_table_final = pd.concat([umr_otm_table, umr_otm_table_double_count], axis=0)

umr_otm_table_final = umr_otm_table_final.reset_index().drop('index',1)


In [167]:
umr_otm_table_final

,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds
0,7_3791139058_Advent Geneva,"[3_3791139058_Credit Suisse, 5_3791139058_Cred..."
1,8_3791139405_Advent Geneva,"[31_3791137060_Northern Trust, 30_3791137060_N..."
2,62_3791139105_Advent Geneva,"[97_3791133104_The Bank of New York Mellon, 2_..."
3,131_3791139278_Advent Geneva,"[50_3791139278_State Street, 48_3791139278_Sta..."
4,52_3791139278_Advent Geneva,"[68_3791137344_State Street, 67_3791137344_Sta..."
5,58_3791139278_Advent Geneva,"[71_3791137344_State Street, 70_3791137344_Sta..."
6,695_379919610_Advent Geneva,[]


In [168]:
# # Before changes on 17-12-2020
# # ### Many to One

# cliff_for_loop = 16

# threshold_1 = X_test['SideA.ViewData.Side1_UniqueIds'].value_counts().reset_index(name='count')
# threshold_1_umb = threshold_1[threshold_1['count']>cliff_for_loop]['index'].unique()
# threshold_1_without_umb = threshold_1[threshold_1['count']<=cliff_for_loop]['index'].unique()

# exceptions_1_umb = X_test[X_test['Predicted_action_2']=='UMB_One_to_One']['SideA.ViewData.Side1_UniqueIds'].value_counts().reset_index(name='count')
# exceptions_1_umb_ids = exceptions_1_umb[exceptions_1_umb['count']>cliff_for_loop]['index'].unique()

# def subSum(numbers,total):
#     for length in range(1, 4):
#         if len(numbers) < length or length < 1:
#             return []
#         for index,number in enumerate(numbers):
#             if length == 1 and np.isclose(number, total,atol=0.25).any():
#                 return [number]
#             subset = subSum(numbers[index+1:],total-number)
#             if subset: 
#                 return [number] + subset
#         return []

# many_ids_0 = []
# one_id_1 = []
# amount_array2 =[]
# for key in X_test[~((X_test['SideA.ViewData.Side1_UniqueIds'].isin(exceptions_1_umb_ids)) |(X_test['SideA.ViewData.Side1_UniqueIds'].isin(final_umr_table['SideA.ViewData.Side1_UniqueIds'])))]['SideA.ViewData.Side1_UniqueIds'].unique():
#     #if key not in ['1174_379879573_State Street','201_379823765_State Street']:
#     print(key)
#     if key in threshold_1_umb:

#         values2 =  X_test[(X_test['SideA.ViewData.Side1_UniqueIds']==key) & (X_test['Predicted_action_2']=='UMB_One_to_One')]['SideB.ViewData.Accounting Net Amount'].values
#         net_sum2 = X_test[X_test['SideA.ViewData.Side1_UniqueIds']==key]['SideA.ViewData.B-P Net Amount'].max()

#         #memo = dict()

#         if subSum(values2,net_sum2) == []: 
#             amount_array2 =[]
#             #print("There are no valid subsets.")

#         else:
#             amount_array2 = subSum(values2,net_sum2)

#             id0_aggregation = X_test[(X_test['SideB.ViewData.Accounting Net Amount'].isin(amount_array2)) & (X_test['SideA.ViewData.Side1_UniqueIds']==key)]['SideB.ViewData.Side0_UniqueIds'].values
#             id1_unique = key       

#             if len(id0_aggregation)>1: 
#                 many_ids_0.append(id0_aggregation)
#                 one_id_1.append(id1_unique)
#             else:
#                 pass

#     else:
#         values2 =  X_test[(X_test['SideA.ViewData.Side1_UniqueIds']==key)]['SideB.ViewData.Accounting Net Amount'].values
#         net_sum2 = X_test[X_test['SideA.ViewData.Side1_UniqueIds']==key]['SideA.ViewData.B-P Net Amount'].max()

#         #memo = dict()

#         if subSum(values2,net_sum2) == []: 
#             amount_array2 =[]
#             #print("There are no valid subsets.")

#         else:
#             amount_array2 = subSum(values2,net_sum2)

#             id0_aggregation = X_test[(X_test['SideB.ViewData.Accounting Net Amount'].isin(amount_array2)) & (X_test['SideA.ViewData.Side1_UniqueIds']==key)]['SideB.ViewData.Side0_UniqueIds'].values
#             id1_unique = key       

#             if len(id0_aggregation)>1: 
#                 many_ids_0.append(id0_aggregation)
#                 one_id_1.append(id1_unique)
#             else:
#                 pass

# umr_mto_table = pd.DataFrame(one_id_1)
# if(umr_mto_table.empty == False):
#     umr_mto_table.columns = ['SideA.ViewData.Side1_UniqueIds']
#     umr_mto_table['SideB.ViewData.Side0_UniqueIds'] =many_ids_0 
# #    umr_mto_table = umr_mto_table[umr_mto_table['SideA.ViewData.Side1_UniqueIds'].isin(one_id_1_final)]
# #    for i in range(0,umr_mto_table.shape[0]):
# #        umr_mto_table['BreakID_Side0'].iloc[i] = list(meo_df[meo_df['ViewData.Side0_UniqueIds'].isin(umr_mto_table['SideB.ViewData.Side0_UniqueIds'].values[i])]['ViewData.BreakID'])
#         #        fun_otm_mto_df['BreakID_Side1'].iloc[i] = list(fun_meo_df[fun_meo_df['ViewData.Side1_UniqueIds'].isin(fun_otm_mto_df['SideA.ViewData.Side1_UniqueIds'].iloc[i])]['ViewData.BreakID'])

# else:
#     temp_umr_mto_table_message = 'No Many to One found'
#     print(temp_umr_mto_table_message)

# # ## Removing duplicate IDs from side 0

# if(len(many_ids_0) == 0):
#     unique_many_ids_0 = ['None']
# else:
#     unique_many_ids_0 = np.unique(np.concatenate(many_ids_0))

# dup_ids_1 = []
# for i in unique_many_ids_0:
#     count =0
#     for j in many_ids_0:
#         if i in j:
#             count = count+1
#             if count==2:
#                 dup_ids_1.append(i)
#                 break             
            
# dup_array_1 = []
# for i in many_ids_0:
#     #print(i)
#     if any(x in dup_ids_1 for x in i):
#         dup_array_1.append(i)
        

# ### Converting array to list
# dup_array_1_list = []
# for i in dup_array_1:
#     dup_array_1_list.append(list(i))
    
# many_ids_0_list =[] 
# for j in many_ids_0:
#     many_ids_0_list.append(list(j))
    
    
# filtered_mto = [i for i in many_ids_0_list if not i in dup_array_1_list]

# one_id_1_final = []
# for i, j in zip(many_ids_0_list, one_id_1):
#     if i in filtered_mto:
#         one_id_1_final.append(j) 


# #pd.set_option('max_columns',50)
# if(umr_mto_table.empty == False):
#     umr_mto_table = umr_mto_table[umr_mto_table['SideA.ViewData.Side1_UniqueIds'].isin(one_id_1_final)]
#     umr_mto_table['BreakID_Side0'] = umr_mto_table.apply(lambda x: list(meo_df[meo_df['ViewData.Side0_UniqueIds'].isin(umr_otm_table_final['SideB.ViewData.Side0_UniqueIds'])]['ViewData.BreakID']), axis=1)
#     for i in range(0,umr_mto_table.shape[0]):
#         umr_mto_table['BreakID_Side0'].iloc[i] = list(meo_df[meo_df['ViewData.Side0_UniqueIds'].isin(umr_mto_table['SideB.ViewData.Side0_UniqueIds'].values[i])]['ViewData.BreakID'])#        fun_otm_mto_df['BreakID_Side1'].iloc[i] = list(fun_meo_df[fun_meo_df['ViewData.Side1_UniqueIds'].isin(fun_otm_mto_df['SideA.ViewData.Side1_UniqueIds'].iloc[i])]['ViewData.BreakID'])

# else:
#     temp_umr_mto_table_message = 'No Many to One found'
#     print(temp_umr_mto_table_message)


# filtered_mto_flat = [item for sublist in filtered_mto for item in sublist]


In [201]:
cliff_for_loop = 17

threshold_1 = X_test['SideA.ViewData.Side1_UniqueIds'].value_counts().reset_index(name='count')
threshold_1_umb = threshold_1[threshold_1['count']>cliff_for_loop]['index'].unique()
threshold_1_without_umb = threshold_1[threshold_1['count']<=cliff_for_loop]['index'].unique()

exceptions_1_umb = X_test[X_test['Predicted_action_2']!='UMR_One_to_One']['SideA.ViewData.Side1_UniqueIds'].value_counts().reset_index(name='count')
exceptions_1_umb_ids = exceptions_1_umb[exceptions_1_umb['count']>cliff_for_loop]['index'].unique()

def subSum(numbers,total):
    for length in range(1, 3):
        if len(numbers) < length or length < 1:
            return []
        for index,number in enumerate(numbers):
            if length == 1 and np.isclose(number, total, atol=0.02).any():
                return [number]
            subset = subSum(numbers[index+1:],total-number)
            if subset: 
                return [number] + subset
        return []
               
#null_value ='No'
many_ids_0 = []
one_id_1 = []
amount_array_2 =[]

loop_count = 0
for key in exceptions_1_umb['index'].unique():
#for key in ['553_1251128974_Advent Geneva','409_1251128952_Advent Geneva']:
    #print(key)
    loop_count = loop_count + 1
    print(loop_count)
    if key in exceptions_1_umb_ids:
        sort_data = X_test[(X_test['SideA.ViewData.Side1_UniqueIds']==key) & (X_test['Predicted_action_2']!='UMR_One_to_One')].sort_values(by = ['probability_UMB_2'], ascending =[False])
        sort_data = sort_data.reset_index().drop('index',1)
        
#        Change made on 08-12-2020. As per Pratik, we will take the first 15 values, as oaktree is smaller data. For weiss, we take 8 as weiss is too big and it takes too much time with value of 15
#        sort_data = sort_data.loc[0:10,:]
        sort_data = sort_data.loc[0:11,:]
        sort_data = sort_data.drop_duplicates(subset=['SideB.ViewData.Accounting Net Amount'])
        sort_data = sort_data.reset_index().drop('index',1)
        #print(sort_data)

        values2 =  sort_data['SideB.ViewData.Accounting Net Amount'].values
        net_sum2 = sort_data['SideA.ViewData.B-P Net Amount'].max()

        #memo = dict()
        #print(values)
        #print(net_sum)

        if subSum(values2,net_sum2) == []: 
            #print("There are no valid subsets.")
            amount_array2 = ['NULL']
        else:
            amount_array2 = subSum(values2,net_sum2)

            id0_aggregation = sort_data[(sort_data['SideB.ViewData.Accounting Net Amount'].isin(amount_array2)) & (sort_data['SideA.ViewData.Side1_UniqueIds']==key)]['SideB.ViewData.Side0_UniqueIds'].values
            id1_unique = key       

            if len(id0_aggregation)>1: 
                many_ids_0.append(id0_aggregation)
                one_id_1.append(id1_unique)
            else:
                pass
            
    else:
        
        sort_data2 = X_test[(X_test['SideA.ViewData.Side1_UniqueIds']==key) & (X_test['Predicted_action_2']!='UMR_One_to_One')].sort_values(by = ['probability_UMB_2'], ascending =[False])
        sort_data2 = sort_data2.reset_index().drop('index',1)
        
#        Change made on 08-12-2020. As per Pratik, we will take the first 8 values, not 10 in order to not overpredict otm and mto umrs
#        sort_data2 = sort_data2.loc[0:10,:]
        sort_data2 = sort_data2.loc[0:11,:]
        sort_data2 = sort_data2.drop_duplicates(subset=['SideB.ViewData.Accounting Net Amount'])
        sort_data2 = sort_data2.reset_index().drop('index',1)
        

        values2 =  sort_data2['SideB.ViewData.Accounting Net Amount'].values
        net_sum2 = sort_data2['SideA.ViewData.B-P Net Amount'].max()
        #values =  X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key) & (X_test['Predicted_action_2']=='UMB_One_to_One')]['SideA.ViewData.B-P Net Amount'].values
        #net_sum = X_test[(X_test['SideB.ViewData.Side0_UniqueIds']==key)& (X_test['Predicted_action_2']=='UMB_One_to_One')]['SideB.ViewData.Accounting Net Amount'].max()

        #memo = dict()
        #print(values)
        #print(net_sum)


        if subSum(values2,net_sum2) == []: 
            #print("There are no valid subsets.")
            amount_array2 = ['NULL']
        else:
            amount_array2 = subSum(values2,net_sum2)

            id0_aggregation = sort_data2[(sort_data2['SideB.ViewData.Accounting Net Amount'].isin(amount_array2)) & (sort_data2['SideA.ViewData.Side1_UniqueIds']==key)]['SideB.ViewData.Side0_UniqueIds'].values
            id1_unique = key       

            if len(id0_aggregation)>1: 
                many_ids_0.append(id0_aggregation)
                one_id_1.append(id1_unique)
            else:
                pass

umr_mto_table = pd.DataFrame(one_id_1)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [202]:
#End change

if umr_mto_table.empty == False:
    umr_mto_table.columns = ['SideA.ViewData.Side1_UniqueIds']
    umr_mto_table['SideB.ViewData.Side0_UniqueIds'] =many_ids_0
else:
    print('No Many to One found')

In [203]:
umr_mto_table

,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Side0_UniqueIds
0,110_3791139278_State Street,"[85_3791139278_Advent Geneva, 84_3791139278_Ad..."
1,109_3791139278_State Street,"[83_3791139278_Advent Geneva, 84_3791139278_Ad..."
2,2_3791139058_Credit Suisse,"[8_3791094530_Advent Geneva, 9_3791094530_Adve..."
3,95_3791137344_State Street,"[39_3791139278_Advent Geneva, 48_3791139278_Ad..."
4,90_3791137344_State Street,"[44_3791139278_Advent Geneva, 35_3791139278_Ad..."
5,89_3791137344_State Street,"[34_3791139278_Advent Geneva, 43_3791139278_Ad..."
6,91_3791137344_State Street,"[36_3791139278_Advent Geneva, 45_3791139278_Ad..."
7,3_379804133_US BANK,"[2_3791044155_Advent Geneva, 1_379919467_Adven..."
8,3_3791139127_JP Morgan,"[23_3791139127_Advent Geneva, 22_3791139127_Ad..."
9,9_3791129289_State Street,"[63_3791114597_Advent Geneva, 61_3791114597_Ad..."


In [223]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='80_3791139278_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,...,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level


In [222]:
meo[meo['ViewData.Side1_UniqueIds']=='46_3791135409_State Street']['ViewData.Side0_UniqueIds'].values[0]

'77_3791122313_Advent Geneva,39_3791123345_Advent Geneva'

In [225]:
meo[meo['ViewData.Side0_UniqueIds']=='80_3791139278_Advent Geneva']

,Unnamed: 0,ViewData.BreakID,ViewData.Status,ViewData.SPM ID,ViewData.Cancel Amount,ViewData.Currency,ViewData.Custodian Account,ViewData.Trade Date,ViewData.Transaction ID,ViewData.Investment ID,...,ViewData.Interest Amount Difference,ViewData.Accounting Trade Expenses,ViewData.Cust Trade Expenses,ViewData.Trade Expenses Difference,ViewData.Accounting OTE Custodian Account,ViewData.Cust OTE Custodian Account,ViewData.OTE Custodian Account Difference,ViewData.Accounting LoanX ID,ViewData.Cust LoanX ID,ViewData.LoanX ID Difference
167,167,1654004372,OB,NaN,-787.13,EUR,OKMF86800402,11-25-2020,6676053,Cognita (Lernen Bidco Ltd) TL-B E+4.25% (FL: 0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
X_test[X_test['SideA.ViewData.Side1_UniqueIds']=='154_3791056034_State Street']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,...,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
4168,1081.85,105.19,0,0,-976.66,8.0,8,1,1,internalinterest,...,SDB,1543331765,UMB_One_to_One,0.197900,0.800895,0.001204,UMB_One_to_One,0.219877,0.780123,0.560246
4169,1081.85,4.03,0,0,-1077.82,8.0,8,1,1,internalinterest,...,SDB,1543331765,UMB_One_to_One,0.203725,0.794967,0.001309,UMB_One_to_One,0.219877,0.780123,0.560246
4170,1081.85,857.28,0,0,-224.57,8.0,8,1,1,internalinterest,...,SDB,1543331765,UMB_One_to_One,0.289461,0.705487,0.005052,UMB_One_to_One,0.426803,0.573197,0.146395
4171,1081.85,384.49,0,0,-697.36,100.0,100,1,1,internalinterest,...,SDB,1543331765,No-Pair,0.760177,0.237948,0.001875,No-Pair,0.618286,0.381714,0.236572
4172,1081.85,-47389.96,0,0,-48471.81,-50.0,-50,1,1,internaldrawdown,...,SDB,1543331765,No-Pair,0.996588,0.003400,0.000012,No-Pair,0.968176,0.031824,0.936352
4173,1081.85,710.85,0,0,-371.00,-50.0,-50,1,1,internalrevenue,...,SDB,1543331765,No-Pair,0.929960,0.069625,0.000415,No-Pair,0.968176,0.031824,0.936352
4174,1081.85,91.82,0,0,-990.03,100.0,100,1,1,internalinterest,...,SDB,1543331765,No-Pair,0.766488,0.231755,0.001757,No-Pair,0.618286,0.381714,0.236572
4175,1081.85,73.26,0,0,-1008.59,8.0,8,1,1,internalinterest,...,SDB,1543331765,UMB_One_to_One,0.197900,0.800895,0.001204,UMB_One_to_One,0.219877,0.780123,0.560246
4176,1081.85,44.73,0,0,-1037.12,8.0,8,1,1,internalinterest,...,SDB,1543331765,UMB_One_to_One,0.208573,0.790193,0.001234,UMB_One_to_One,0.219877,0.780123,0.560246
4177,1081.85,710.85,0,0,-371.00,-67.0,-67,1,1,internalrevenue,...,SDB,1543331765,No-Pair,0.765608,0.233194,0.001198,No-Pair,0.966997,0.033003,0.933993


In [86]:
umr_mto_table.loc[0,'SideB.ViewData.Side0_UniqueIds']

array(['609_3791044365_Advent Geneva', '610_3791044365_Advent Geneva',
       '614_3791044365_Advent Geneva', '615_3791044365_Advent Geneva',
       '352_3791044365_Advent Geneva', '616_3791044365_Advent Geneva',
       '150_3791151084_Advent Geneva'], dtype=object)

In [87]:
umr_mto_table.loc[1,'SideB.ViewData.Side0_UniqueIds']

array(['614_3791044365_Advent Geneva', '615_3791044365_Advent Geneva',
       '352_3791044365_Advent Geneva', '150_3791151084_Advent Geneva',
       '644_379919610_Advent Geneva'], dtype=object)

In [88]:
umr_mto_table.loc[2,'SideB.ViewData.Side0_UniqueIds']

array(['641_379919610_Advent Geneva', '649_379919610_Advent Geneva',
       '748_379919610_Advent Geneva', '644_379919610_Advent Geneva',
       '640_379919610_Advent Geneva', '647_379919610_Advent Geneva',
       '648_379919610_Advent Geneva', '148_3791151084_Advent Geneva'],
      dtype=object)

In [206]:


# ## Removing duplicate IDs from side 0

if len(many_ids_0)!=0:
    unique_many_ids_0 = np.unique(np.concatenate(many_ids_0))
else:
    unique_many_ids_0 = np.array(['None'])

dup_ids_1 = []
for i in unique_many_ids_0:
    count =0
    for j in many_ids_0:
        if i in j:
            count = count+1
            if count==2:
                dup_ids_1.append(i)
                break             
            
dup_array_1 = []
for i in many_ids_0:
    #print(i)
    if any(x in dup_ids_1 for x in i):
        dup_array_1.append(i)
 
### Converting array to list
dup_array_1_list = []
for i in dup_array_1:
    dup_array_1_list.append(list(i))
    
many_ids_0_list =[] 
for j in many_ids_0:
    many_ids_0_list.append(list(j))
    
    
filtered_mto = [i for i in many_ids_0_list if not i in dup_array_1_list]

one_id_1_final = []
for i, j in zip(many_ids_0_list, one_id_1):
    if i in filtered_mto:
        one_id_1_final.append(j) 

if len(one_id_1_final)!=0:
    #unique_many_ids_1 = np.unique(np.concatenate(many_ids_1))
    one_id_1_final = one_id_1_final
else:
    one_id_1_final = np.array(['None'])

#umr_otm_table = umr_otm_table[umr_otm_table['SideB.ViewData.Side0_UniqueIds'].isin(one_id_0_final)]
#umr_mto_table = umr_mto_table[umr_mto_table['SideA.ViewData.Side1_UniqueIds'].isin(one_id_1_final)]
#umr_mto_table = umr_mto_table.reset_index().drop('index',1)


if(umr_mto_table.empty == False):
    umr_mto_table = umr_mto_table[umr_mto_table['SideA.ViewData.Side1_UniqueIds'].isin(one_id_1_final)]
    umr_mto_table = umr_mto_table.reset_index().drop('index',1)
#TODO : Revisit this code later - start here
#    umr_mto_table['BreakID_Side0'] = umr_mto_table.apply(lambda x: list(meo_df[meo_df['ViewData.Side0_UniqueIds'].isin(umr_mto_table['SideB.ViewData.Side0_UniqueIds'])]['ViewData.BreakID']), axis=1)
#    for i in range(0,umr_mto_table.shape[0]):
#        umr_mto_table['BreakID_Side0'].iloc[i] = list(meo_df[meo_df['ViewData.Side0_UniqueIds'].isin(umr_mto_table['SideB.ViewData.Side0_UniqueIds'].values[i])]['ViewData.BreakID'])#        fun_otm_mto_df['BreakID_Side1'].iloc[i] = list(fun_meo_df[fun_meo_df['ViewData.Side1_UniqueIds'].isin(fun_otm_mto_df['SideA.ViewData.Side1_UniqueIds'].iloc[i])]['ViewData.BreakID'])
#TODO : Revisit this code later - end here
else:
    temp_umr_mto_table_message = 'No Many to One found'
    print(temp_umr_mto_table_message)






In [207]:
umr_mto_table

,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Side0_UniqueIds
0,2_3791139058_Credit Suisse,"[8_3791094530_Advent Geneva, 9_3791094530_Adve..."
1,95_3791137344_State Street,"[39_3791139278_Advent Geneva, 48_3791139278_Ad..."
2,90_3791137344_State Street,"[44_3791139278_Advent Geneva, 35_3791139278_Ad..."
3,89_3791137344_State Street,"[34_3791139278_Advent Geneva, 43_3791139278_Ad..."
4,91_3791137344_State Street,"[36_3791139278_Advent Geneva, 45_3791139278_Ad..."
5,3_379804133_US BANK,"[2_3791044155_Advent Geneva, 1_379919467_Adven..."
6,3_3791139127_JP Morgan,"[23_3791139127_Advent Geneva, 22_3791139127_Ad..."
7,86_3791137344_State Street,"[33_3791139278_Advent Geneva, 24_3791139278_Ad..."
8,85_3791137344_State Street,"[32_3791139278_Advent Geneva, 23_3791139278_Ad..."
9,15_3791133038_BNP Paribas,"[25_3791135288_Advent Geneva, 3_3791139052_Adv..."


In [174]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='44_3791150882_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,...,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level


In [175]:
filtered_mto_flat = [item for sublist in filtered_mto for item in sublist]


In [176]:


# ## Removing all the OTM and MTO Ids

X_test_left2 = X_test_left[~(X_test_left['SideB.ViewData.Side0_UniqueIds'].isin(filtered_mto_flat))]

X_test_left2 = X_test_left2[~(X_test_left2['SideA.ViewData.Side1_UniqueIds'].isin(list(one_id_1)))]

X_test_left2 = X_test_left[~(X_test_left['SideB.ViewData.Side0_UniqueIds'].isin(filtered_otm_flat))]
X_test_left2 = X_test_left2[~(X_test_left2['SideB.ViewData.Side0_UniqueIds'].isin(list(one_id_0)))]

X_test_left2 = X_test_left2.reset_index().drop('index',1)

# ## UMB one to one (final)

X_test_umb = X_test_left2[X_test_left2['Predicted_action_2']=='UMB_One_to_One']
X_test_umb = X_test_umb.reset_index().drop('index',1)

one_side_unique_umb_ids = one_to_one_umb(X_test_umb)

final_oto_umb_table = X_test_umb[X_test_umb['SideA.ViewData.Side1_UniqueIds'].isin(one_side_unique_umb_ids)]

final_oto_umb_table = final_oto_umb_table[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side','Predicted_action_2','probability_No_pair_2','probability_UMB_2','probability_UMR']]

final_oto_umb_table['probability_UMR'] = 0.00010
final_oto_umb_table = final_oto_umb_table.rename(columns = {'Predicted_action_2':'Predicted_action','probability_No_pair_2':'probability_No_pair','probability_UMB_2':'probability_UMB'})

# ## Removing IDs from OTO UMB

X_test_left3 = X_test_left2[~(X_test_left2['SideB.ViewData.Side0_UniqueIds'].isin(final_oto_umb_table['SideB.ViewData.Side0_UniqueIds']))]
X_test_left3 = X_test_left3[~(X_test_left3['SideA.ViewData.Side1_UniqueIds'].isin(final_oto_umb_table['SideA.ViewData.Side1_UniqueIds']))]


X_test_left3 = X_test_left3.reset_index().drop('index',1)

# ## UMB One to Many and Many to One

## Total IDs 

X_test['SideB.ViewData.Side0_UniqueIds'].nunique() + X_test['SideA.ViewData.Side1_UniqueIds'].nunique()
X_test_left3['SideB.ViewData.Side0_UniqueIds'].nunique() + X_test_left3['SideA.ViewData.Side1_UniqueIds'].nunique()

open_ids_0_last , open_ids_1_last = no_pair_seg2(X_test_left3)

X_test_left3[~X_test_left3['SideB.ViewData.Side0_UniqueIds'].isin(open_ids_0_last)]

X_test_left4 = X_test_left3[~((X_test_left3['SideB.ViewData.Side0_UniqueIds'].isin(open_ids_0_last)) | (X_test_left3['SideA.ViewData.Side1_UniqueIds'].isin(open_ids_1_last)))]

X_test_left4 = X_test_left4.reset_index().drop('index',1)
X_test_left4['SideB.ViewData.Side0_UniqueIds'].nunique() + X_test_left4['SideA.ViewData.Side1_UniqueIds'].nunique()

# ## Many to Many new

#MANY TO MANY NEW
rr2 = X_test[X_test['Predicted_action_2']=='UMB_One_to_One'].groupby(['SideB.ViewData.Side0_UniqueIds'])['SideA.ViewData.Side1_UniqueIds'].unique().reset_index()
rr2['SideA.ViewData.Side1_UniqueIds'] = rr2['SideA.ViewData.Side1_UniqueIds'].apply(tuple)

rr2.groupby(['SideA.ViewData.Side1_UniqueIds'])['SideB.ViewData.Side0_UniqueIds'].unique().reset_index()

rr2 = X_test_left4[X_test_left4['Predicted_action_2']=='UMB_One_to_One'].groupby(['SideB.ViewData.Side0_UniqueIds'])['SideA.ViewData.Side1_UniqueIds'].unique().reset_index()

acc_amount = X_test[X_test['Predicted_action_2']=='UMB_One_to_One'].groupby(['SideB.ViewData.Side0_UniqueIds'])['SideB.ViewData.Accounting Net Amount'].max().reset_index()
pb_amount_sum =  X_test[X_test['Predicted_action_2']=='UMB_One_to_One'].groupby(['SideB.ViewData.Side0_UniqueIds'])['SideA.ViewData.B-P Net Amount'].sum().reset_index()

rr3 = pd.merge(rr2, acc_amount, on='SideB.ViewData.Side0_UniqueIds', how='left')
rr4 = pd.merge(rr3, pb_amount_sum, on='SideB.ViewData.Side0_UniqueIds', how='left')

rr4['SideA.ViewData.Side1_UniqueIds'] = rr4['SideA.ViewData.Side1_UniqueIds'].apply(tuple)

rr5 = rr4.groupby(['SideA.ViewData.Side1_UniqueIds'])['SideB.ViewData.Side0_UniqueIds'].unique().reset_index()

rr6 = pd.merge(rr5, rr4.groupby(['SideA.ViewData.Side1_UniqueIds'])['SideB.ViewData.Accounting Net Amount'].sum().reset_index(), on='SideA.ViewData.Side1_UniqueIds', how='left')

rr7 = pd.merge(rr6,rr4[['SideA.ViewData.Side1_UniqueIds','SideA.ViewData.B-P Net Amount']].drop_duplicates(), on='SideA.ViewData.Side1_UniqueIds',how='left')

rr7['diff'] = rr7['SideB.ViewData.Accounting Net Amount'] - rr7['SideA.ViewData.B-P Net Amount']

rr7['pb_len'] = rr7['SideA.ViewData.Side1_UniqueIds'].apply(lambda x: len(x))
rr7['acc_len'] = rr7['SideB.ViewData.Side0_UniqueIds'].apply(lambda x: len(x))

rr8 = rr7[~((rr7['pb_len']==1)|(rr7['acc_len']==1))]
rr8 = rr8.reset_index().drop('index',1)



In [177]:
rr8

,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Side0_UniqueIds,SideB.ViewData.Accounting Net Amount,SideA.ViewData.B-P Net Amount,diff,pb_len,acc_len
0,"(100_3791127028_The Bank of New York Mellon, 9...","[108_3791127028_Advent Geneva, 109_3791127028_...",24789.54,27061.81,-2272.27,3,5
1,"(11_3791057814_Credit Suisse, 10_3791057814_Cr...","[10_3791052102_Advent Geneva, 11_3791052102_Ad...",132220.03,-3131118.77,3263338.80,10,7
2,"(15_3791133038_BNP Paribas, 14_3791133038_BNP ...","[25_3791135288_Advent Geneva, 3_3791139052_Adv...",123085.00,125031.85,-1946.85,2,2
3,"(15_3791139405_Northern Trust, 12_3791139405_N...","[24_3791125244_Advent Geneva, 32_3791133117_Ad...",382491.32,3836.84,378654.48,2,2
4,"(1_3791017949_State Street, 1_3791044365_State...","[49_3791048390_Advent Geneva, 50_3791048390_Ad...",-0.01,35882.36,-35882.37,4,2
5,"(1_3791017949_State Street, 1_3791044365_State...","[49_3791048390_Advent Geneva, 50_3791048390_Ad...",-0.01,37132.36,-37132.37,4,2
6,"(239_3791001204_State Street, 33_3791017949_St...","[44_3791011889_Advent Geneva, 62_3791114597_Ad...",-41180.46,-28735360.04,28694179.58,8,2
7,"(239_3791001204_State Street, 33_3791017949_St...","[44_3791011889_Advent Geneva, 62_3791114597_Ad...",-41180.46,-68054056.90,68012876.44,8,2
8,"(28_379939728_Credit Suisse, 29_379939728_Cred...","[10_3791094530_Advent Geneva, 11_3791094530_Ad...",-9511446.27,-65089706.17,55578259.90,10,8
9,"(28_379939728_Credit Suisse, 29_379939728_Cred...","[5_3791094530_Advent Geneva, 6_3791094530_Adve...",9738015.47,-65016753.61,74754769.08,12,2


## Settle date MTM

In [179]:
cc = pd.concat([aa, bb], axis=0)

cc = cc.reset_index().drop('index',1)

cc['ViewData.Transaction Type'] = cc['ViewData.Transaction Type'].astype(str)
cc['ViewData.Settle Date'] = pd.to_datetime(cc['ViewData.Settle Date'])
cc['filter_key_with_sd'] = cc['filter_key'].astype(str) + cc['ViewData.Settle Date'].astype(str)


##########################
cc7 = cc.copy()

cc_new = cc7[cc7['ViewData.Status']!='SPM']
cc_new = cc_new.reset_index().drop('index',1)

#cc_new = cc_new[~((cc_new['ViewData.Side0_UniqueIds'].isin(final_umr_table['SideB.ViewData.Side0_UniqueIds'])) | (cc_new['ViewData.Side1_UniqueIds'].isin(final_umr_table['SideA.ViewData.Side1_UniqueIds'])))]

#cc_new = cc_new[~((cc_new['ViewData.Side0_UniqueIds'].isin(final_umt_table['SideB.ViewData.Side0_UniqueIds'])) | (cc_new['ViewData.Side1_UniqueIds'].isin(final_umt_table['SideA.ViewData.Side1_UniqueIds'])))]

cc_new = cc_new.reset_index().drop('index',1)

filter_key_umt_umb_sd = []
diff_in_amount_sd = []
diff_in_amount_key_sd = []
for key in cc_new['filter_key_with_sd'].unique():    
    cc_dummy = cc_new[cc_new['filter_key_with_sd']==key]
    if (-0.25<= cc_dummy['ViewData.Net Amount Difference'].sum() <=0.25) & (cc_dummy.shape[0]>2) & (cc_dummy['Trans_side'].nunique()>1):
        #print(cc2_dummy.shape[0])
        #print(key)
        filter_key_umt_umb_sd.append(key)
    else:
        if (cc_dummy.shape[0]>2) & (cc_dummy['Trans_side'].nunique()>1):
            diff = cc_dummy['ViewData.Net Amount Difference'].sum()
            diff_in_amount_sd.append(diff)
            diff_in_amount_key_sd.append(key)

## Equity Swap Many to many

sd_mtm_1_ids = []
sd_mtm_0_ids = []

for key in filter_key_umt_umb_sd:
    one_side = cc_new[cc_new['filter_key_with_sd']== key]['ViewData.Side1_UniqueIds'].unique()
    zero_side = cc_new[cc_new['filter_key_with_sd']== key]['ViewData.Side0_UniqueIds'].unique()
    one_side = [i for i in one_side if i not in ['nan','None','']]
    zero_side = [i for i in zero_side if i not in ['nan','None','']]
    sd_mtm_1_ids.append(one_side)
    sd_mtm_0_ids.append(zero_side)

if sd_mtm_1_ids !=[]:
    sd_mtm_list_1 = list(np.concatenate(sd_mtm_1_ids))
else:
    sd_mtm_list_1 = []

if sd_mtm_0_ids !=[]:
    sd_mtm_list_0 = list(np.concatenate(sd_mtm_0_ids))
else:
    sd_mtm_list_0 = []

## Data Frame for MTM from equity Swap

mtm_df_sd = pd.DataFrame(np.arange(len(sd_mtm_0_ids)))
mtm_df_sd.columns = ['index']

mtm_df_sd['ViewData.Side0_UniqueIds'] = sd_mtm_0_ids
mtm_df_sd['ViewData.Side1_UniqueIds'] = sd_mtm_1_ids
mtm_df_sd = mtm_df_sd.drop('index',1)

In [181]:
mtm_df_sd

,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds
0,"[45_3791139278_Advent Geneva, 44_3791139278_Ad...","[93_3791137344_State Street, 87_3791137344_Sta..."
1,"[23_3791139278_Advent Geneva, 24_3791139278_Ad...","[85_3791137344_State Street, 86_3791137344_Sta..."
2,"[47_3791137344_Advent Geneva, 16_3791139278_Ad...","[6_3791135409_State Street, 5_3791135409_State..."
3,"[52_3791139278_Advent Geneva, 53_3791139278_Ad...","[67_3791137344_State Street, 68_3791137344_Sta..."
4,"[58_3791139278_Advent Geneva, 59_3791139278_Ad...","[70_3791137344_State Street, 71_3791137344_Sta..."
5,"[14_3791139052_Advent Geneva, 15_3791139052_Ad...","[8_3791139052_BNP Paribas, 9_3791139052_BNP Pa..."
